In [ ]:
# 🚑 Fix for "Numpy is not available" in Colab

!pip install --upgrade --force-reinstall numpy==1.26.4
# (Optional) Also reinstall other core packages if needed:
# !pip install --upgrade --force-reinstall torch transformers pandas

print("✅ Numpy reinstalled. Now RESTART the runtime (Runtime > Restart runtime) before continuing.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.2.0 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.4.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
✅ Numpy reinstalled. Now RESTART the runtime (Runtime > Restart runtime) before continuing.


In [ ]:
# Cell 1: Install Ollama and Start Server

# 1. Download and install Ollama
# This script automatically downloads the correct version for your Colab environment (Linux)
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama server in the background
# We use nohup to ensure it runs even if the cell output is cleared.
# We also redirect output to a file and run it in the background (&).
!nohup ollama serve > ollama.log 2>&1 &

# 3. Give Ollama a moment to start
import time
print("Waiting for Ollama server to start...")
time.sleep(10) # Give it 10 seconds to fully initialize
print("Ollama server should be running.")

# (Optional) Verify Ollama is running by checking its process
!ps aux | grep ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Waiting for Ollama server to start...
Ollama server should be running.
root        3154  1.8  0.0 6828428 43116 ?       Sl   16:36   0:00 ollama serve
root        3208  0.0  0.0   7376  3504 ?        S    16:36   0:00 /bin/bash -c ps aux | grep ollama
root        3210  0.0  0.0   6484  2272 ?        S    16:36   0:00 grep ollama


In [ ]:
!ollama list

# If nothing shows up, pull the model
!ollama pull llama3.2

# Check again
!ollama list

# If you want to see more details about what's happening
!ollama show llama3.2

NAME    ID    SIZE    MODIFIED 

NAME               ID              SIZE      MODIFIED               
llama3.2:latest    a80c4f17acd5    2.0 GB    Less than a second ago    
  Model
    architecture        llama     
    parameters          3.2B      
    context length      131072    
    embedding length    3072      
    quantization        Q4_K_M    

  Capabilities
    completion    
    tools         

  Parameters
    stop    "<|start_header_id|>"    
    stop    "<|end_header_id|>"      
    stop    "<|eot_id|>"             

  License
    LLAMA 3.2 COMMUNITY LICENSE AGREEMENT                 
    Llama 3.2 Version Release Date: September 25, 2024    
    ...                                                   



In [ ]:
# Cell 1: Install Ollama and Start Server
from google.colab import drive
drive.mount('/content/drive')

# 1. Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama server in the background
!nohup ollama serve > ollama.log 2>&1 &

# 3. Give Ollama a moment to start
import time
print("Waiting for Ollama server to start...")
time.sleep(10)
print("Ollama server should be running.")

# Verify Ollama is running
!ps aux | grep ollama

# Cell 2: Pull Llama Model
# Pull the Llama model we want to use
!ollama pull llama3.2

print("✅ Llama 3.2 model downloaded successfully!")

"""
TRUE BASE GENERATION SYSTEM - LOCAL OLLAMA VERSION
No external knowledge sources - pure LLM generation from text only
All external APIs, Knowledge Graphs, and Location enrichment DISABLED
Uses LOCAL Ollama instead of HuggingFace authentication
"""

import os
import re
import time
import logging
import requests
import json
from typing import List, Dict, Any

# Configuration
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"
OUTPUT_BASE_TTL = '/content/drive/MyDrive/extracted_events_base_generation_ollama.ttl'
OLLAMA_BASE_URL = "http://localhost:11434"

# TRUE BASE GENERATION FLAGS - All external knowledge DISABLED
RAG_ENABLED = False
KNOWLEDGE_GRAPHS_ENABLED = False
LOCATION_ENRICHMENT_ENABLED = False
EXTERNAL_APIS_ENABLED = False

# Token limits
MAX_TOKENS_PER_REQUEST = 100000
CHUNK_OVERLAP = 200

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class OllamaLLM:
    """Local Ollama LLM wrapper"""

    def __init__(self, model_name: str = "llama3.2", base_url: str = OLLAMA_BASE_URL):
        self.model_name = model_name
        self.base_url = base_url
        self.generate_url = f"{base_url}/api/generate"

        print(f"🦙 Initializing Local Ollama with model: {model_name}")

        # Test connection to Ollama
        try:
            response = requests.get(f"{base_url}/api/tags", timeout=10)
            if response.status_code == 200:
                models = response.json().get('models', [])
                model_names = [m['name'] for m in models]
                print(f"✅ Connected to Ollama. Available models: {model_names}")

                if any(model_name in name for name in model_names):
                    print(f"✅ Model {model_name} is available!")
                else:
                    print(f"⚠️ Model {model_name} not found. Attempting to pull...")
                    self.pull_model()
            else:
                raise Exception(f"Ollama server returned status {response.status_code}")

        except Exception as e:
            print(f"❌ Error connecting to Ollama: {e}")
            print("💡 Make sure Ollama server is running: ollama serve")
            raise

    def pull_model(self):
        """Pull the model if it's not available"""
        try:
            print(f"🔄 Pulling model {self.model_name}...")
            pull_url = f"{self.base_url}/api/pull"
            response = requests.post(pull_url, json={"name": self.model_name}, timeout=300)

            if response.status_code == 200:
                print(f"✅ Successfully pulled model {self.model_name}")
            else:
                print(f"❌ Failed to pull model: {response.text}")

        except Exception as e:
            print(f"❌ Error pulling model: {e}")

    def generate(self, prompt: str, **kwargs) -> str:
        """Generate response using Ollama"""
        try:
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": kwargs.get("temperature", 0.7),
                    "num_predict": kwargs.get("max_length", 200),
                    "top_p": kwargs.get("top_p", 0.9)
                }
            }

            response = requests.post(
                self.generate_url,
                json=payload,
                timeout=120  # 2 minute timeout
            )

            if response.status_code == 200:
                result = response.json()
                return result.get("response", "")
            else:
                logger.error(f"Ollama request failed: {response.status_code} - {response.text}")
                return ""

        except Exception as e:
            logger.error(f"Error generating with Ollama: {e}")
            return ""

class TextChunker:
    """Handles text chunking to manage token limits"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 1000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class TrueBaseGenerationSystem:
    """TRUE Base Generation System - No external knowledge sources"""

    def __init__(self):
        self.chunker = TextChunker()
        self.stats = {
            'chunks_processed': 0,
            'events_extracted': 0,
            'external_api_calls': 0,  # This will stay 0
            'knowledge_sources_used': 0,  # This will stay 0
        }
        logger.info("TRUE BASE GENERATION SYSTEM INITIALIZED")
        logger.info("❌ Knowledge Graphs: DISABLED")
        logger.info("❌ Location Enrichment: DISABLED")
        logger.info("❌ External APIs: DISABLED")
        logger.info("❌ RAG Text Retrieval: DISABLED")
        logger.info("✅ Pure LLM Generation: ENABLED")

    def extract_basic_entities_from_text_only(self, text: str) -> List[str]:
        """Extract entities using ONLY pattern matching - no external validation"""
        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in matches:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:5]

    def extract_basic_locations_from_text_only(self, text: str) -> List[str]:
        """Extract locations using ONLY pattern matching - no coordinate lookup"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and not loc.isdigit()):
                filtered_locations.append(loc)

        return list(set(filtered_locations))[:3]

    def process_chunk_true_base(self, chunk: str, chunk_num: int, llm) -> str:
        """Process chunk with TRUE base generation - no external knowledge"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars) - TRUE BASE GENERATION")

        entities = self.extract_basic_entities_from_text_only(chunk)
        locations = self.extract_basic_locations_from_text_only(chunk)

        logger.info(f"Found entities (text-only): {entities[:3]}...")
        logger.info(f"Found locations (text-only): {locations[:3]}...")

        if not entities and not locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        base_prompt = f"""You are extracting historical events from text using ONLY the information provided in the text chunk. Do not use external knowledge sources, but you CAN and SHOULD make reasonable inferences from the text.

TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

ENTITIES FOUND IN TEXT: {', '.join(entities) if entities else 'None'}
LOCATIONS FOUND IN TEXT: {', '.join(locations) if locations else 'None'}

TASK: Extract historical events mentioned in this text chunk using the text information and making REASONABLE INFERENCES.

REQUIREMENTS:
1. Extract ONLY events explicitly mentioned in the text chunk
2. Use information directly stated in the text
3. MAKE REASONABLE INFERENCES from context clues in the text
4. If you can reasonably infer coordinates, countries, regions from textual context, DO IT
5. Include ALL these properties for each event:
   - ste:hasType (description of event, enhanced with context)
   - ste:hasAgent (who caused/led the event, with inferred roles)
   - ste:hasTime (when it happened, with inferred specificity)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (infer approximate coordinates if you can from text context)
   - ste:hasLongitude (infer approximate coordinates if you can from text context)
   - ste:hasCountry (infer country from textual geographic context)
   - ste:hasRegion (infer region from textual geographic context)
   - ste:hasLocationSource "inferred" (if you made geographic inferences)
   - ste:hasResult (outcome, enhanced with contextual inference)

INFERENCE GUIDELINES:
- If text mentions "Athens", infer it's in Greece, approximate coordinates
- If text mentions "Sicily", infer it's in Italy, Mediterranean coordinates
- If text mentions "Sparta/Lacedaemon", infer Peloponnese, Greece
- If you know from context clues what geographic region events occurred in, infer coordinates
- If someone is called "King", infer royal title
- If text implies timeframes, infer more specific dates
- If outcomes are implied, infer likely results

Output format (do not include prefixes):
```turtle
ste:Event{chunk_num}_1 a ste:Event ;
    ste:hasType "specific event type inferred from context" ;
    ste:hasAgent "person/group with inferred roles" ;
    ste:hasTime "time period with inferred specificity" ;
    ste:hasLocation "location name from text" ;
    ste:hasLatitude "37.9838" ;
    ste:hasLongitude "23.7275" ;
    ste:hasCountry "Greece" ;
    ste:hasRegion "Attica" ;
    ste:hasLocationSource "inferred" ;
    ste:hasResult "outcome inferred from context" .
```

CRITICAL:
- Use the SAME output format as enhanced systems for fair comparison
- INFER coordinates, countries, regions if you can reasonably deduce them from text
- Make the events as detailed and specific as possible through inference
- If you truly cannot infer something, then use empty string ""
- The goal is to extract maximum information through text analysis and inference

If no clear historical events are mentioned in the text, return empty.
"""

        try:
            # Generate response using Ollama
            generated_text = llm.generate(
                base_prompt,
                temperature=0.7,
                max_length=400
            )

            # DEBUG: Print what the model actually generated
            print(f"🔍 Raw model output for chunk {chunk_num}:")
            print(f"'{generated_text[:200]}...'")

            turtle_output = self.clean_turtle(generated_text)

            # DEBUG: Print cleaned output
            print(f"🔍 Cleaned turtle output:")
            print(f"'{turtle_output[:200]}...'")

            if turtle_output:
                self.stats['chunks_processed'] += 1
                event_count = turtle_output.count('ste:Event')
                self.stats['events_extracted'] += event_count
                logger.info(f"Generated {event_count} events from chunk {chunk_num}")
            else:
                print(f"⚠️ No turtle output after cleaning for chunk {chunk_num}")

            return turtle_output

        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num}: {e}")
            return ""

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        # Look for turtle code blocks first
        import re
        turtle_match = re.search(r'```(?:turtle)?\s*(.*?)```', raw_output, re.DOTALL | re.IGNORECASE)
        if turtle_match:
            raw_output = turtle_match.group(1)

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('ste:') or
                stripped.startswith('a ') or
                ':' in stripped and ('hasType' in stripped or 'hasAgent' in stripped or 'hasTime' in stripped)):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

# Utility functions
def load_api_key():
    """Ollama doesn't need an API key"""
    print("✅ Using Local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"❌ File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"✅ Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"❌ Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Local Ollama"""
    try:
        print("🦙 Initializing Local Ollama...")
        llm = OllamaLLM(model_name="llama3.2")
        print("✅ Local Ollama initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing Ollama: {e}")
        print("💡 Make sure Ollama server is running: ollama serve")
        print("💡 And the model is pulled: ollama pull llama3.2")
        return None

def main():
    """Main function - TRUE BASE GENERATION WITH LOCAL OLLAMA"""
    print("🚀 Starting TRUE BASE GENERATION SYSTEM with Local Ollama")
    print("="*60)
    print("🦙 LLM: Local Ollama (No Authentication Required)")
    print("❌ Knowledge Graphs: DISABLED")
    print("❌ Location Coordinate Lookup: DISABLED")
    print("❌ External APIs: DISABLED")
    print("❌ RAG Text Retrieval: DISABLED")
    print("❌ All External Knowledge Sources: DISABLED")
    print("✅ Pure LLM Generation from Text Only: ENABLED")
    print("="*60)

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    base_system = TrueBaseGenerationSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    token_count = base_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    # Use smaller chunks for Ollama
    if token_count > 2000:  # Reduced for Ollama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = base_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Process chunks with TRUE base generation
    all_turtle_outputs = []

    print("\n🔄 Processing chunks with TRUE BASE GENERATION...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = base_system.process_chunk_true_base(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        if i < len(chunks):
            time.sleep(2)  # Pause between chunks for Ollama

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

"""

        final_output = prefixes + "# TRUE BASE GENERATION - No External Knowledge Sources (Local Ollama)\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_BASE_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved TRUE BASE GENERATION RDF to {OUTPUT_BASE_TTL}")
        print(f"📊 TRUE BASE GENERATION Statistics:")
        print(f"   - LLM: Local Ollama (No Authentication)")
        print(f"   - Generation Mode: PURE BASE (No External Knowledge)")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Events extracted: {base_system.stats['events_extracted']}")

        print(f"\n📝 Sample of TRUE BASE GENERATION RDF:")
        print("="*60)
        print(final_output[:400] + "..." if len(final_output) > 400 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    print(f"\n🎉 TRUE BASE GENERATION complete!")
    print(f"📄 Output file: {OUTPUT_BASE_TTL}")

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Waiting for Ollama server to start...
Ollama server should be running.
root       32227  1.7  0.0 6827888 44068 ?       Sl   09:47   0:00 ollama serve
root       32283  0.0  0.0   7376  3516 ?        S    09:47   0:00 /bin/bash -c ps aux | grep ollama
root       32285  0.0  0.0   6484  2192 ?        S    09:47   0:00 grep ollama

✅ Llama 3.2 model downloaded successfully!
🚀 Starting TRUE BASE GENERATION SYSTEM with Local Ollam

In [ ]:
# Cell 2: Pull the llama3.2 model

print("Pulling llama3.2 model (this may take a while)...")
# You can check the progress in the output of this cell
!ollama pull llama3.2

print("\nModel 'llama3.2' should now be available.")
# (Optional) List installed models to confirm
!ollama list

Pulling llama3.2 model (this may take a while)...


Model 'llama3.2' should now be available.
NAME               ID              SIZE      MODIFIED               
llama3.2:latest    a80c4f17acd5    2.0 GB    Less than a second ago    
llama3:latest      365c0bd3c000    4.7 GB    25 minutes ago            


In [ ]:
import numpy as np
print("✅ Numpy is available:", np.__version__)


✅ Numpy is available: 1.26.4


In [ ]:

# Cell 1: Mount Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# --- Dependency Compatibility Guard ---
import os

# Force compatible versions for transformers and sentence-transformers
os.system("pip install transformers==4.40.2 sentence-transformers==2.4.0 --quiet")

# (Optional) If you use torch, ensure it's compatible too:
# os.system("pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu --quiet")

# Now safe to import

!pip install  torch accelerate rdflib


"""
Enhanced Multi-Knowledge Graph RAG System with Text Chunking - LLAMA VERSION
Handles large texts by processing them in chunks to avoid token limits
"""

import os
import re
import time
import json
import numpy as np
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal # <--- This line is crucial
from rdflib.namespace import XSD, SKOS
# Add a direct import test here to diagnose immediately
print("\nVerifying Langchain Ollama import...")
try:
    from langchain_ollama import ChatOllama
    print("✅ ChatOllama successfully imported after reinstallation.")
except ImportError as e:
    print(f"❌ CRITICAL ERROR: ChatOllama still cannot be imported after reinstallation: {e}")
    print("Please check your pip installation output for errors and ensure your Colab runtime is healthy.")
    # You might want to exit here if this is a persistent issue
    # import sys
    # sys.exit(1)
except Exception as e:
    print(f"❌ An unexpected error occurred during ChatOllama import verification: {e}")
    # import sys
    # sys.exit(1)


# Configuration
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"
ONTOLOGY_PATH = "/content/drive/MyDrive/wiki.owl"
LOCATION_ONTOLOGY_PATH = "/content/drive/MyDrive/locations.owl"
OUTPUT_RAG_TTL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl'
OUTPUT_RAG_OWL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.owl'
KG_CACHE_FILE = '/content/drive/MyDrive/kg_cache.json'
LOCATION_CACHE_FILE = '/content/drive/MyDrive/location_cache.json'
KG_ANALYSIS_REPORT = '/content/drive/MyDrive/multi_kg_analysis_report.txt'

# Token limits - adjusted for Llama
MAX_TOKENS_PER_REQUEST = 50000  # Conservative limit for Llama
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_ollama import ChatOllama  # Changed from OpenAI to Ollama
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("pip install rdflib python-dotenv langchain langchain-ollama langchain-community faiss-cpu sentence-transformers requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""

    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()

    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}

    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")

    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None

    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)

        return list(set(filtered_locations))

    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None

        try:
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """

            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")

        return None

    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])

                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )

        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")

        return None

    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata with disambiguation"""
        try:
            time.sleep(0.5)

            # Try multiple query strategies to get the right location
            queries = [
                # Try exact label match first
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """,
                # Try with additional filters for places/locations
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  ?item wdt:P31/wdt:P279* wd:Q486972 .  # human settlement
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """
            ]

            for query in queries:
                params = {'query': query, 'format': 'json'}
                response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    bindings = data.get('results', {}).get('bindings', [])

                    if bindings:
                        # Prefer results with country information
                        best_binding = None
                        for binding in bindings:
                            if binding.get('country'):
                                best_binding = binding
                                break

                        if not best_binding:
                            best_binding = bindings[0]

                        coord_str = best_binding.get('coord', {}).get('value', '')

                        coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                        if coord_match:
                            longitude = float(coord_match.group(1))
                            latitude = float(coord_match.group(2))

                            return LocationInfo(
                                name=location_name,
                                latitude=latitude,
                                longitude=longitude,
                                country=best_binding.get('countryLabel', {}).get('value', ''),
                                source="wikidata",
                                uri=best_binding.get('item', {}).get('value', '')
                            )

        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")

        return None

    def validate_coordinates(self, location_info: LocationInfo) -> bool:
        """Validate that coordinates make sense for the location"""
        if not location_info.latitude or not location_info.longitude:
            return True

        lat, lon = location_info.latitude, location_info.longitude

        # Basic coordinate range validation
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            logger.warning(f"Invalid coordinates for {location_info.name}: {lat}, {lon}")
            return False

        return True

    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None

        location_info = None

        location_info = self.get_location_from_ontology(location_name)

        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)

        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)

        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None

        self._save_location_cache()

        if location_info:
            self.validate_coordinates(location_info)

        return location_info

class TextChunker:
    """Handles text chunking to manage token limits for Llama"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count for Llama (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 10000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""

    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0

    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""

    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata with timeout protection"""
        try:
            self._rate_limit_wait()

            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}

              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)

              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
            else:
                logger.warning(f"Wikidata returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"Wikidata query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"Wikidata query failed for '{entity}': {e}")

        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""

    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia with timeout protection"""
        try:
            self._rate_limit_wait()

            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')

                    predicate_name = predicate.split('/')[-1].replace('_', ' ')

                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
            else:
                logger.warning(f"DBpedia returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"DBpedia query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"DBpedia query failed for '{entity}': {e}")

        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector with dynamic concept discovery"""

    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)

    def search_related_concepts(self, entity: str) -> List[str]:
        """Search for related concepts using ConceptNet's search API"""
        try:
            # Try search API first
            search_url = f"{self.base_url}/search?text={entity.replace(' ', '%20')}&limit=10"
            response = requests.get(search_url, timeout=10)

            related_concepts = []
            if response.status_code == 200:
                data = response.json()
                for edge in data.get('edges', []):
                    start = edge.get('start', {}).get('label', '')
                    end = edge.get('end', {}).get('label', '')

                    # Extract concept paths and clean them
                    for concept_path in [start, end]:
                        if concept_path and '/c/en/' in concept_path:
                            concept = concept_path.replace('/c/en/', '').replace('_', ' ')
                            if concept.lower() != entity.lower() and len(concept) > 2:
                                related_concepts.append(concept)

            return list(set(related_concepts))[:5]  # Return top 5 unique concepts

        except Exception as e:
            logger.debug(f"ConceptNet search failed for {entity}: {e}")
            return []

    def query_concept_directly(self, concept: str, limit: int = 20) -> List[dict]:
        """Query a specific concept and return raw edges"""
        try:
            concept_path = f"/c/en/{concept.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept_path}?limit={limit}"

            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data.get('edges', [])

        except Exception as e:
            logger.debug(f"ConceptNet direct query failed for {concept}: {e}")

        return []

    def retrieve_facts(self, entity: str, limit: int = 100) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet through dynamic discovery"""
        try:
            self._rate_limit_wait()
            all_facts = []

            # Strategy 1: Try direct query first
            direct_edges = self.query_concept_directly(entity, limit//2)

            # Strategy 2: Search for related concepts and query them
            related_concepts = self.search_related_concepts(entity)

            # Process direct edges
            for edge in direct_edges:
                fact = self._edge_to_fact(edge, entity, "direct")
                if fact:
                    all_facts.append(fact)

            # Process related concept edges
            for concept in related_concepts:
                concept_edges = self.query_concept_directly(concept, 5)
                for edge in concept_edges:
                    fact = self._edge_to_fact(edge, entity, f"via_{concept}")
                    if fact:
                        all_facts.append(fact)

            if all_facts:
                self.success_count += 1
                logger.info(f"Retrieved {len(all_facts)} facts from ConceptNet for '{entity}'")
                if related_concepts:
                    logger.info(f"  - Found related concepts: {related_concepts}")

            return all_facts[:limit]

        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")

        return []

    def _edge_to_fact(self, edge: dict, original_entity: str, discovery_method: str) -> Optional[EnhancedKnowledgeFact]:
        """Convert ConceptNet edge to EnhancedKnowledgeFact"""
        try:
            start = edge.get('start', {})
            end = edge.get('end', {})
            relation = edge.get('rel', {})
            weight = edge.get('weight', 1.0)

            start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
            end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
            rel_label = relation.get('label', 'related_to')

            # Skip if labels are empty or too short
            if not start_label or not end_label or len(start_label) < 2 or len(end_label) < 2:
                return None

            # Determine confidence based on discovery method
            confidence_multiplier = 1.0 if discovery_method == "direct" else 0.6

            return EnhancedKnowledgeFact(
                subject=original_entity,
                predicate=rel_label,
                object=end_label if start_label.lower() in original_entity.lower() else start_label,
                source=self.name,
                confidence=min(weight * confidence_multiplier, 1.0),
                context=f"Discovered {discovery_method}"
            )

        except Exception as e:
            logger.debug(f"Error converting edge to fact: {e}")
            return None

class MultiKGCache:
    """Caching system for knowledge graph facts"""

    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}

    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")

    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"

    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)

    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph RAG system with chunking and location extraction - LLAMA VERSION"""

    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.global_locations = {}
        self.vectorstore = None  # For RAG retrieval
        self.document_chunks = []  # Store processed chunks for RAG
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0,
            'location_duplicates_avoided': 0,
            'rag_queries': 0
        }

    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []

        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:12]

    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs with improved timeout handling"""
        all_facts = {}
        cache_hits = 0

        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}

            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 5)
                        futures[future] = (entity, source_name)

            # Collect results with better timeout handling
            completed = 0
            total_futures = len(futures)

            try:
                for future in as_completed(futures, timeout=45):  # Increased timeout
                    entity, source_name = futures[future]
                    completed += 1

                    try:
                        facts = future.result(timeout=5)  # Individual future timeout
                        if facts:
                            self.cache.set(source_name, entity, facts)

                            if entity not in all_facts:
                                all_facts[entity] = []
                            all_facts[entity].extend(facts)

                            self.stats['facts_retrieved'] += len(facts)

                        logger.debug(f"✅ {source_name} completed for {entity} ({completed}/{total_futures})")

                    except Exception as e:
                        logger.warning(f"❌ {source_name} failed for {entity}: {e}")
                        continue

            except TimeoutError:
                pending_count = total_futures - completed
                logger.warning(f"⏰ Timeout: {pending_count}/{total_futures} KG queries still pending, continuing with available results")

                # Cancel remaining futures
                for future in futures:
                    if not future.done():
                        future.cancel()

        self.stats['cache_hits'] += cache_hits
        logger.info(f"KG retrieval completed: {completed}/{total_futures} successful, {cache_hits} cache hits")
        return all_facts

    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []

        for entity, facts in kg_facts.items():
            if facts:
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)

                context_parts.append(f"\n=== Knowledge about {entity} ===")

                by_source = {}
                for fact in sorted_facts[:4]:
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)

                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:2]:
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)

        return "\n".join(context_parts)

    def register_global_location(self, location_info: LocationInfo) -> str:
        """Register location globally and return unique identifier"""
        location_key = location_info.name.lower().strip()

        if location_key in self.global_locations:
            existing = self.global_locations[location_key]
            if (location_info.latitude and location_info.longitude and
                (not existing.latitude or not existing.longitude)):
                self.global_locations[location_key] = location_info
                logger.info(f"Updated coordinates for {location_info.name}")
            else:
                self.stats['location_duplicates_avoided'] += 1
                logger.debug(f"Location {location_info.name} already registered")
        else:
            self.global_locations[location_key] = location_info
            logger.info(f"Registered new location: {location_info.name}")

        clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
        return f"ste:Location_{clean_name}"

    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text with RAG-enhanced location extraction"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars)")

        # 1. RAG RETRIEVAL: Get relevant context from vector store
        relevant_context = ""
        if self.vectorstore:
            try:
                # Use the chunk as a query to retrieve similar/relevant text
                relevant_docs = self.vectorstore.similarity_search(chunk, k=7)
                retrieved_chunks = [doc.page_content for doc in relevant_docs]
                relevant_context = "\n---\n".join(retrieved_chunks)
                logger.info(f"Retrieved {len(retrieved_chunks)} relevant chunks via RAG for chunk {chunk_num}")
            except Exception as e:
                logger.warning(f"RAG retrieval failed for chunk {chunk_num}: {e}")
                relevant_context = ""

        # 2. Extract entities and locations
        entities = self.extract_entities_advanced(chunk)
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")

        # 3. Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:5]:  # Limit to 5 to avoid overwhelming
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                self.register_global_location(location_info)
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1

        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")

        # 4. Get KG facts for entities
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        location_context = self.format_location_context(enriched_locations)

        # 5. RAG-ENHANCED PROMPT: Use retrieved context + KG facts + locations
        enhanced_prompt = f"""You are extracting historical events using RAG (Retrieval-Augmented Generation). Use ALL available context sources to enhance your extraction.

CURRENT TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

RAG RETRIEVED RELEVANT CONTEXT:
{relevant_context if relevant_context else "No relevant context retrieved."}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in the current text chunk. Use the RAG retrieved context, KG facts, and location coordinates to enhance details but stay faithful to what's actually in the current chunk.

Requirements:
1. Extract ONLY events mentioned in the CURRENT text chunk (not from retrieved context)
2. Use RAG retrieved context to provide additional historical context and validation
3. Use KG facts to enhance entity information
4. Use location coordinates to provide precise geographical data
5. Include ALL these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasRegion (region if available)
   - ste:hasLocationSource (source of coordinates: wikidata/dbpedia/local_ontology)
   - ste:hasResult (outcome/consequence)
   - ste:hasRAGContext "yes" (to indicate this was RAG-enhanced)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from current chunk" ;
    ste:hasAgent "specific person from current chunk" ;
    ste:hasTime "specific date from current chunk" ;
    ste:hasLocation "specific location from current chunk" ;
    ste:hasLatitude "37.1234"^^xsd:double ;
    ste:hasLongitude "15.5678"^^xsd:double ;
    ste:hasCountry "Italy" ;
    ste:hasRegion "Sicily" ;
    ste:hasLocationSource "wikidata" ;
    ste:hasResult "specific outcome from current chunk" ;
    ste:hasRAGContext "yes" .
```

IMPORTANT:
- The PRIMARY source is the CURRENT text chunk - extract events from IT
- Use RAG retrieved context to validate and enhance your understanding
- Use KG facts to enrich entity details
- Include precise coordinates from location sources
- Mark all events with ste:hasRAGContext "yes" to show RAG was used
- Only extract events explicitly mentioned in the current chunk
- If no clear events are found in current chunk, return empty
"""

        try:
            response = llm.invoke([HumanMessage(content=enhanced_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            self.stats['rag_queries'] += 1  # Count as RAG usage
            logger.info(f"Generated RAG-enhanced RDF for chunk {chunk_num}")
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num} with RAG: {e}")
            return ""

    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."

        context_parts = ["\n=== Location Information ==="]

        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")

            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
            else:
                context_parts.append("  - Coordinates: Not available")

            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")

            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")

            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")

        return "\n".join(context_parts)

    def generate_global_location_rdf(self) -> str:
        """Generate RDF for all unique locations found across all chunks"""
        if not self.global_locations:
            return ""

        location_rdf_parts = []

        for location_key, location_info in self.global_locations.items():
            clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
            location_id = f"ste:Location_{clean_name}"

            rdf_lines = [f'{location_id} a ste:Location ;']
            rdf_lines.append(f'    rdfs:label "{location_info.name}" ;')

            if location_info.latitude and location_info.longitude:
                rdf_lines.append(f'    geo:lat "{location_info.latitude}"^^xsd:double ;')
                rdf_lines.append(f'    geo:long "{location_info.longitude}"^^xsd:double ;')

            if location_info.country:
                rdf_lines.append(f'    ste:hasCountry "{location_info.country}" ;')

            if location_info.region:
                rdf_lines.append(f'    ste:hasRegion "{location_info.region}" ;')

            if location_info.source:
                rdf_lines.append(f'    ste:hasSource "{location_info.source}" ;')

            if location_info.uri:
                rdf_lines.append(f'    ste:hasURI <{location_info.uri}> ;')

            if rdf_lines[-1].endswith(' ;'):
                rdf_lines[-1] = rdf_lines[-1][:-2] + ' .'

            location_rdf_parts.append('\n'.join(rdf_lines))

        return '\n\n'.join(location_rdf_parts)

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or
                stripped.startswith(':') or stripped.startswith('_') or
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

    def prepare_vectorstore(self, text_chunks: List[str]):
        """Create vector store from text chunks for RAG retrieval"""
        try:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            from langchain_community.vectorstores import FAISS

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            )

            # Prepare chunks with metadata
            documents = []
            metadatas = []

            for i, chunk in enumerate(text_chunks):
                if len(chunk.strip()) > 50:  # Only include substantial chunks
                    documents.append(chunk)
                    metadatas.append({
                        'chunk_id': i,
                        'length': len(chunk),
                        'type': 'text_chunk'
                    })

            if documents:
                self.vectorstore = FAISS.from_texts(documents, embeddings, metadatas=metadatas)
                self.document_chunks = documents
                logger.info(f"Created vector store with {len(documents)} chunks")
                return True
            else:
                logger.warning("No suitable chunks found for vector store")
                return False

        except Exception as e:
            logger.error(f"Error creating vector store: {e}")
            return False

    def rag_query(self, query: str, llm, k: int = 3) -> Dict[str, Any]:
        """Perform RAG query: retrieve relevant chunks + KG facts, then generate response"""
        self.stats['rag_queries'] += 1
        logger.info(f"Processing RAG query: '{query[:50]}...'")

        if not self.vectorstore:
            return {"error": "Vector store not initialized. Call prepare_vectorstore() first."}

        try:
            # 1. RETRIEVE: Get relevant text chunks
            relevant_docs = self.vectorstore.similarity_search(query, k=k)
            retrieved_chunks = [doc.page_content for doc in relevant_docs]

            # 2. EXTRACT: Get entities from query
            query_entities = self.extract_entities_advanced(query)

            # 3. RETRIEVE: Get KG facts for entities
            kg_facts = self.retrieve_kg_facts_enhanced(query_entities)
            kg_context = self.format_kg_context_enhanced(kg_facts)

            # 4. EXTRACT & ENRICH: Get locations from query
            query_locations = self.location_extractor.extract_locations_from_text(query)
            enriched_locations = {}
            for location_name in query_locations[:5]:
                location_info = self.location_extractor.enrich_location(location_name)
                if location_info:
                    enriched_locations[location_name] = location_info

            location_context = self.format_location_context(enriched_locations)

            # 5. AUGMENT: Create enhanced context for generation
            context_parts = [
                f"QUERY: {query}",
                f"\nRETRIEVED RELEVANT TEXT CHUNKS:",
                "\n" + "\n---\n".join(retrieved_chunks),
                f"\nKNOWLEDGE GRAPH CONTEXT:",
                kg_context,
                f"\nLOCATION CONTEXT:",
                location_context
            ]

            enhanced_context = "\n".join(context_parts)

            # 6. GENERATE: Create comprehensive response
            rag_prompt = f"""You are an expert historian with access to multiple knowledge sources. Answer the question using the provided context.

{enhanced_context}

INSTRUCTIONS:
1. Answer the question comprehensively using ALL available context
2. Cite specific information from the retrieved text chunks
3. Incorporate relevant knowledge graph facts to enhance your answer
4. Include precise location information and coordinates when relevant
5. If information conflicts between sources, mention this
6. Be specific about dates, places, people, and events
7. If you cannot answer completely, explain what information is missing

QUESTION: {query}

ANSWER:"""

            # 7. GENERATE: Get LLM response
            response = llm.invoke([HumanMessage(content=rag_prompt)])

            return {
                "query": query,
                "answer": response.content,
                "retrieved_chunks": retrieved_chunks,
                "entities_found": query_entities,
                "kg_facts_count": sum(len(facts) for facts in kg_facts.values()),
                "locations_found": list(enriched_locations.keys()),
                "sources": {
                    "text_chunks": len(retrieved_chunks),
                    "kg_sources": list(set(fact.source for facts in kg_facts.values() for fact in facts)),
                    "location_sources": list(set(loc.source for loc in enriched_locations.values()))
                }
            }

        except Exception as e:
            logger.error(f"RAG query failed: {e}")
            return {"error": str(e)}

    def interactive_rag_session(self, llm):
        """Start an interactive RAG session"""
        print("\n🤖 Starting Interactive RAG Session")
        print("Ask questions about your text. Type 'quit' to exit.")
        print("=" * 60)

        while True:
            try:
                query = input("\n❓ Your question: ").strip()

                if query.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye!")
                    break

                if not query:
                    continue

                print("\n🔍 Processing your question...")
                result = self.rag_query(query, llm)

                if "error" in result:
                    print(f"❌ Error: {result['error']}")
                    continue

                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")

            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue

# Utility functions
def load_api_key():
    """Llama via Ollama doesn't need an API key"""
    print("✅ Using local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Llama LLM"""
    try:
        llm = ChatOllama(model="llama3.2", temperature=0)
        print("✅ Llama LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        print("💡 Make sure Ollama is running: ollama serve")
        print("💡 And pull the model: ollama pull llama3.2")
        return None

def prepare_vectorstore_from_text(text: str, multi_kg_system):
    """Create vector store from text"""
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        from langchain_community.vectorstores import FAISS

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )

        # Split text into sentences for better retrieval
        sentences = re.split(r'[.!?]+', text)
        texts = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 20]

        if not texts:
            return None

        vectorstore = FAISS.from_texts(texts, embeddings)
        multi_kg_system.vectorstore = vectorstore
        multi_kg_system.document_chunks = texts
        print(f"📚 Vector store created with {len(texts)} text segments")
        return vectorstore
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def main():
    """Main function with chunking support and RAG capabilities - LLAMA VERSION"""
    print("🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)")

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    # Prepare vector store for RAG FIRST
    print("\n📚 Setting up RAG vector store...")
    vectorstore = prepare_vectorstore_from_text(domain_text, multi_kg_system)

    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    if token_count > 10000:  # Reduced for Llama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Extract events and create RDF
    all_turtle_outputs = []
    all_entities = set()

    print("\n🔄 Processing chunks for event extraction...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)

        if i < len(chunks):
            time.sleep(1)

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""

        final_output = prefixes + "# Historical Events with RAG-Enhanced Embedded Location Data (LLAMA)\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved enhanced RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        print(f"   - Location duplicates avoided: {multi_kg_system.stats['location_duplicates_avoided']}")
        print(f"   - Unique global locations: {len(multi_kg_system.global_locations)}")
        print(f"   - RAG queries for RDF generation: {multi_kg_system.stats['rag_queries']}")

        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")

        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")

        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    # START RAG SESSION HERE!
    if vectorstore:
        print(f"\n🤖 RAG System Ready!")

        # Show example queries
        print(f"\n💡 Try asking questions like:")
        print(f"   - 'What battles happened in Sicily?'")
        print(f"   - 'Who were the main leaders mentioned?'")
        print(f"   - 'What events occurred in 415 BC?'")
        print(f"   - 'Describe the naval engagements'")
        print(f"   - 'What was the outcome of the siege?'")

        # Ask if user wants to start interactive session
        response = input(f"\n❓ Start interactive RAG session? (y/n): ").strip()

        # Check if user typed a question instead of y/n
        if response.lower() not in ['y', 'yes', 'n', 'no', '']:
            # User typed a question directly!
            print(f"\n🔍 Processing your question: '{response}'")
            result = multi_kg_system.rag_query(response, llm)

            if "error" not in result:
                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")
            else:
                print(f"❌ Error: {result['error']}")

            # Ask if they want to continue with interactive session
            continue_response = input(f"\n❓ Continue with interactive RAG session? (y/n): ").strip().lower()
            if continue_response in ['y', 'yes', '']:
                multi_kg_system.interactive_rag_session(llm)

        elif response.lower() in ['y', 'yes', '']:
            multi_kg_system.interactive_rag_session(llm)
        else:
            print(f"\n💡 You can also query programmatically:")
            print(f"   result = multi_kg_system.rag_query('your question', llm)")

            # Offer a few sample queries
            sample_queries = [
                "What are the main events mentioned in the text?",
                "Which locations are mentioned?",
                "Who are the key people involved?"
            ]

            print(f"\n🔍 Running sample queries:")
            for query in sample_queries:
                print(f"\n❓ Sample query: '{query}'")
                result = multi_kg_system.rag_query(query, llm)
                if "error" not in result:
                    print(f"📝 Answer: {result['answer'][:200]}...")
                    print(f"📊 Sources: {len(result['retrieved_chunks'])} chunks, {result['kg_facts_count']} KG facts")
                else:
                    print(f"❌ Error: {result['error']}")
                print("-" * 40)

    else:
        print("⚠️  Could not create vector store for RAG functionality")

    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for RDF results.")

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Verifying Langchain Ollama import...
✅ ChatOllama successfully imported after reinstallation.
🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)
✅ Using local Ollama - no API key needed.
Loaded text from /content/drive/MyDrive/part_aa
📄 Using YOUR text from /content/drive/MyDrive/part_aa
📝 Text length: 398568 characters
✅ Llama LLM initialized successfully.

📚 Setting up RAG vector store...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📚 Vector store created with 1980 text segments
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 10 chunks

🔄 Processing chunks for event extraction...

🔄 Processing chunk 1/10...


This
This
Author
Author
Translator
Translator
Release Date
Release Date



🔄 Processing chunk 2/10...

🔄 Processing chunk 3/10...


...


🔄 Processing chunk 4/10...

🔄 Processing chunk 5/10...

🔄 Processing chunk 6/10...

🔄 Processing chunk 7/10...


Second Year
Second Year



🔄 Processing chunk 8/10...


Third Year
Third Year



🔄 Processing chunk 9/10...

🔄 Processing chunk 10/10...

✅ Saved enhanced RDF to /content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl
📊 Processing Statistics:
   - Total chunks processed: 10
   - Successful chunks: 10
   - Unique entities found: 97
   - Total KG facts retrieved: 2
   - Cache hits: 254
   - Locations found: 21
   - Locations with coordinates: 21
   - Location duplicates avoided: 12
   - Unique global locations: 9
   - RAG queries for RDF generation: 10

🔗 Knowledge Graph Connector Statistics:
   - Wikidata: 26/34 requests (76.5% success)
   - DBpedia: 21/29 requests (72.4% success)
   - ConceptNet: 1/43 requests (2.3% success)
   - Location enrichment: 55/178 locations enriched (30.9% success)

📝 Sample of generated RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbp

In [ ]:
# 🚑 Fix sentence-transformers/transformers incompatibility

# 1. Uninstall all possibly conflicting versions
!pip uninstall -y sentence-transformers transformers

# 2. Install compatible versions
!pip install transformers==4.40.2
!pip install sentence-transformers==2.4.0

# 3. Test import
try:
    from sentence_transformers import SentenceTransformer
    print("✅ sentence_transformers import now works!")
except ImportError as e:
    print(f"❌ sentence_transformers import failed: {e}")

import transformers
print(f"transformers version: {transformers.__version__}")


Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: transformers 4.52.3
Uninstalling transformers-4.52.3:
  Successfully uninstalled transformers-4.52.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.2.0 requires sentence-transformers>=2.6.0, which is not installed.
  Using cached sentence_transformers-2.4.0-


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

✅ sentence_transformers import now works!
transformers version: 4.40.2


In [ ]:

# Cell 1: Mount Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# --- Dependency Compatibility Guard ---
import os

# Force compatible versions for transformers and sentence-transformers
os.system("pip install transformers==4.40.2 sentence-transformers==2.4.0 --quiet")

# (Optional) If you use torch, ensure it's compatible too:
# os.system("pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu --quiet")

# Now safe to import

!pip install  torch accelerate rdflib


"""
Enhanced Multi-Knowledge Graph RAG System with Text Chunking - LLAMA VERSION
Handles large texts by processing them in chunks to avoid token limits
"""

import os
import re
import time
import json
import numpy as np
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal # <--- This line is crucial
from rdflib.namespace import XSD, SKOS
# Add a direct import test here to diagnose immediately
print("\nVerifying Langchain Ollama import...")
try:
    from langchain_ollama import ChatOllama
    print("✅ ChatOllama successfully imported after reinstallation.")
except ImportError as e:
    print(f"❌ CRITICAL ERROR: ChatOllama still cannot be imported after reinstallation: {e}")
    print("Please check your pip installation output for errors and ensure your Colab runtime is healthy.")
    # You might want to exit here if this is a persistent issue
    # import sys
    # sys.exit(1)
except Exception as e:
    print(f"❌ An unexpected error occurred during ChatOllama import verification: {e}")
    # import sys
    # sys.exit(1)


# Configuration
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"
ONTOLOGY_PATH = "/content/drive/MyDrive/wiki.owl"
LOCATION_ONTOLOGY_PATH = "/content/drive/MyDrive/locations.owl"
OUTPUT_RAG_TTL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama_1.ttl'
OUTPUT_RAG_OWL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama_1.owl'
KG_CACHE_FILE = '/content/drive/MyDrive/kg_cache.json'
LOCATION_CACHE_FILE = '/content/drive/MyDrive/location_cache.json'
KG_ANALYSIS_REPORT = '/content/drive/MyDrive/multi_kg_analysis_report.txt'

# Token limits - adjusted for Llama
MAX_TOKENS_PER_REQUEST = 50000  # Conservative limit for Llama
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_ollama import ChatOllama  # Changed from OpenAI to Ollama
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("pip install rdflib python-dotenv langchain langchain-ollama langchain-community faiss-cpu sentence-transformers requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""

    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()

    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}

    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")

    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None

    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)

        return list(set(filtered_locations))

    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None

        try:
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """

            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")

        return None

    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])

                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )

        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")

        return None

    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata with disambiguation"""
        try:
            time.sleep(0.5)

            # Try multiple query strategies to get the right location
            queries = [
                # Try exact label match first
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """,
                # Try with additional filters for places/locations
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  ?item wdt:P31/wdt:P279* wd:Q486972 .  # human settlement
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """
            ]

            for query in queries:
                params = {'query': query, 'format': 'json'}
                response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    bindings = data.get('results', {}).get('bindings', [])

                    if bindings:
                        # Prefer results with country information
                        best_binding = None
                        for binding in bindings:
                            if binding.get('country'):
                                best_binding = binding
                                break

                        if not best_binding:
                            best_binding = bindings[0]

                        coord_str = best_binding.get('coord', {}).get('value', '')

                        coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                        if coord_match:
                            longitude = float(coord_match.group(1))
                            latitude = float(coord_match.group(2))

                            return LocationInfo(
                                name=location_name,
                                latitude=latitude,
                                longitude=longitude,
                                country=best_binding.get('countryLabel', {}).get('value', ''),
                                source="wikidata",
                                uri=best_binding.get('item', {}).get('value', '')
                            )

        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")

        return None

    def validate_coordinates(self, location_info: LocationInfo) -> bool:
        """Validate that coordinates make sense for the location"""
        if not location_info.latitude or not location_info.longitude:
            return True

        lat, lon = location_info.latitude, location_info.longitude

        # Basic coordinate range validation
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            logger.warning(f"Invalid coordinates for {location_info.name}: {lat}, {lon}")
            return False

        return True

    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None

        location_info = None

        location_info = self.get_location_from_ontology(location_name)

        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)

        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)

        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None

        self._save_location_cache()

        if location_info:
            self.validate_coordinates(location_info)

        return location_info

class TextChunker:
    """Handles text chunking to manage token limits for Llama"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count for Llama (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 10000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""

    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0

    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }

    def retrieve_facts(self, entity: str, limit: int = 2) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""

    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 2) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata with timeout protection"""
        try:
            self._rate_limit_wait()

            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}

              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)

              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
            else:
                logger.warning(f"Wikidata returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"Wikidata query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"Wikidata query failed for '{entity}': {e}")

        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""

    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia with timeout protection"""
        try:
            self._rate_limit_wait()

            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')

                    predicate_name = predicate.split('/')[-1].replace('_', ' ')

                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
            else:
                logger.warning(f"DBpedia returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"DBpedia query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"DBpedia query failed for '{entity}': {e}")

        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector with dynamic concept discovery"""

    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)

    def search_related_concepts(self, entity: str) -> List[str]:
        """Search for related concepts using ConceptNet's search API"""
        try:
            # Try search API first
            search_url = f"{self.base_url}/search?text={entity.replace(' ', '%20')}&limit=10"
            response = requests.get(search_url, timeout=10)

            related_concepts = []
            if response.status_code == 200:
                data = response.json()
                for edge in data.get('edges', []):
                    start = edge.get('start', {}).get('label', '')
                    end = edge.get('end', {}).get('label', '')

                    # Extract concept paths and clean them
                    for concept_path in [start, end]:
                        if concept_path and '/c/en/' in concept_path:
                            concept = concept_path.replace('/c/en/', '').replace('_', ' ')
                            if concept.lower() != entity.lower() and len(concept) > 2:
                                related_concepts.append(concept)

            return list(set(related_concepts))[:5]  # Return top 5 unique concepts

        except Exception as e:
            logger.debug(f"ConceptNet search failed for {entity}: {e}")
            return []

    def query_concept_directly(self, concept: str, limit: int = 20) -> List[dict]:
        """Query a specific concept and return raw edges"""
        try:
            concept_path = f"/c/en/{concept.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept_path}?limit={limit}"

            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data.get('edges', [])

        except Exception as e:
            logger.debug(f"ConceptNet direct query failed for {concept}: {e}")

        return []

    def retrieve_facts(self, entity: str, limit: int = 40) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet through dynamic discovery"""
        try:
            self._rate_limit_wait()
            all_facts = []

            # Strategy 1: Try direct query first
            direct_edges = self.query_concept_directly(entity, limit//2)

            # Strategy 2: Search for related concepts and query them
            related_concepts = self.search_related_concepts(entity)

            # Process direct edges
            for edge in direct_edges:
                fact = self._edge_to_fact(edge, entity, "direct")
                if fact:
                    all_facts.append(fact)

            # Process related concept edges
            for concept in related_concepts:
                concept_edges = self.query_concept_directly(concept, 5)
                for edge in concept_edges:
                    fact = self._edge_to_fact(edge, entity, f"via_{concept}")
                    if fact:
                        all_facts.append(fact)

            if all_facts:
                self.success_count += 1
                logger.info(f"Retrieved {len(all_facts)} facts from ConceptNet for '{entity}'")
                if related_concepts:
                    logger.info(f"  - Found related concepts: {related_concepts}")

            return all_facts[:limit]

        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")

        return []

    def _edge_to_fact(self, edge: dict, original_entity: str, discovery_method: str) -> Optional[EnhancedKnowledgeFact]:
        """Convert ConceptNet edge to EnhancedKnowledgeFact"""
        try:
            start = edge.get('start', {})
            end = edge.get('end', {})
            relation = edge.get('rel', {})
            weight = edge.get('weight', 1.0)

            start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
            end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
            rel_label = relation.get('label', 'related_to')

            # Skip if labels are empty or too short
            if not start_label or not end_label or len(start_label) < 2 or len(end_label) < 2:
                return None

            # Determine confidence based on discovery method
            confidence_multiplier = 1.0 if discovery_method == "direct" else 0.6

            return EnhancedKnowledgeFact(
                subject=original_entity,
                predicate=rel_label,
                object=end_label if start_label.lower() in original_entity.lower() else start_label,
                source=self.name,
                confidence=min(weight * confidence_multiplier, 1.0),
                context=f"Discovered {discovery_method}"
            )

        except Exception as e:
            logger.debug(f"Error converting edge to fact: {e}")
            return None

class MultiKGCache:
    """Caching system for knowledge graph facts"""

    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}

    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")

    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"

    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)

    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph RAG system with chunking and location extraction - LLAMA VERSION"""

    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.global_locations = {}
        self.vectorstore = None  # For RAG retrieval
        self.document_chunks = []  # Store processed chunks for RAG
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0,
            'location_duplicates_avoided': 0,
            'rag_queries': 0
        }

    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []

        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:12]

    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs with improved timeout handling"""
        all_facts = {}
        cache_hits = 0

        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}

            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 5)
                        futures[future] = (entity, source_name)

            # Collect results with better timeout handling
            completed = 0
            total_futures = len(futures)

            try:
                for future in as_completed(futures, timeout=45):  # Increased timeout
                    entity, source_name = futures[future]
                    completed += 1

                    try:
                        facts = future.result(timeout=5)  # Individual future timeout
                        if facts:
                            self.cache.set(source_name, entity, facts)

                            if entity not in all_facts:
                                all_facts[entity] = []
                            all_facts[entity].extend(facts)

                            self.stats['facts_retrieved'] += len(facts)

                        logger.debug(f"✅ {source_name} completed for {entity} ({completed}/{total_futures})")

                    except Exception as e:
                        logger.warning(f"❌ {source_name} failed for {entity}: {e}")
                        continue

            except TimeoutError:
                pending_count = total_futures - completed
                logger.warning(f"⏰ Timeout: {pending_count}/{total_futures} KG queries still pending, continuing with available results")

                # Cancel remaining futures
                for future in futures:
                    if not future.done():
                        future.cancel()

        self.stats['cache_hits'] += cache_hits
        logger.info(f"KG retrieval completed: {completed}/{total_futures} successful, {cache_hits} cache hits")
        return all_facts

    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []

        for entity, facts in kg_facts.items():
            if facts:
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)

                context_parts.append(f"\n=== Knowledge about {entity} ===")

                by_source = {}
                for fact in sorted_facts[:2]:
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)

                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:2]:
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)

        return "\n".join(context_parts)

    def register_global_location(self, location_info: LocationInfo) -> str:
        """Register location globally and return unique identifier"""
        location_key = location_info.name.lower().strip()

        if location_key in self.global_locations:
            existing = self.global_locations[location_key]
            if (location_info.latitude and location_info.longitude and
                (not existing.latitude or not existing.longitude)):
                self.global_locations[location_key] = location_info
                logger.info(f"Updated coordinates for {location_info.name}")
            else:
                self.stats['location_duplicates_avoided'] += 1
                logger.debug(f"Location {location_info.name} already registered")
        else:
            self.global_locations[location_key] = location_info
            logger.info(f"Registered new location: {location_info.name}")

        clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
        return f"ste:Location_{clean_name}"

    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text with RAG-enhanced location extraction"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars)")

        # 1. RAG RETRIEVAL: Get relevant context from vector store
        relevant_context = ""
        if self.vectorstore:
            try:
                # Use the chunk as a query to retrieve similar/relevant text
                relevant_docs = self.vectorstore.similarity_search(chunk, k=4)
                retrieved_chunks = [doc.page_content for doc in relevant_docs]
                relevant_context = "\n---\n".join(retrieved_chunks)
                logger.info(f"Retrieved {len(retrieved_chunks)} relevant chunks via RAG for chunk {chunk_num}")
            except Exception as e:
                logger.warning(f"RAG retrieval failed for chunk {chunk_num}: {e}")
                relevant_context = ""

        # 2. Extract entities and locations
        entities = self.extract_entities_advanced(chunk)
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")

        # 3. Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:3]:  # Limit to 5 to avoid overwhelming
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                self.register_global_location(location_info)
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1

        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")

        # 4. Get KG facts for entities
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        location_context = self.format_location_context(enriched_locations)

        # 5. RAG-ENHANCED PROMPT: Use retrieved context + KG facts + locations
        enhanced_prompt = f"""You are extracting historical events using RAG (Retrieval-Augmented Generation). Use ALL available context sources to enhance your extraction.

CURRENT TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

RAG RETRIEVED RELEVANT CONTEXT:
{relevant_context if relevant_context else "No relevant context retrieved."}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in the current text chunk. Use the RAG retrieved context, KG facts, and location coordinates to enhance details but stay faithful to what's actually in the current chunk.

Requirements:
1. Extract ONLY events mentioned in the CURRENT text chunk (not from retrieved context)
2. Use RAG retrieved context to provide additional historical context and validation
3. Use KG facts to enhance entity information
4. Use location coordinates to provide precise geographical data
5. Include ALL these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasRegion (region if available)
   - ste:hasLocationSource (source of coordinates: wikidata/dbpedia/local_ontology)
   - ste:hasResult (outcome/consequence)
   - ste:hasRAGContext "yes" (to indicate this was RAG-enhanced)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from current chunk" ;
    ste:hasAgent "specific person from current chunk" ;
    ste:hasTime "specific date from current chunk" ;
    ste:hasLocation "specific location from current chunk" ;
    ste:hasLatitude "37.1234"^^xsd:double ;
    ste:hasLongitude "15.5678"^^xsd:double ;
    ste:hasCountry "Italy" ;
    ste:hasRegion "Sicily" ;
    ste:hasLocationSource "wikidata" ;
    ste:hasResult "specific outcome from current chunk" ;
    ste:hasRAGContext "yes" .
```

IMPORTANT:
- The PRIMARY source is the CURRENT text chunk - extract events from IT
- Use RAG retrieved context to validate and enhance your understanding
- Use KG facts to enrich entity details
- Include precise coordinates from location sources
- Mark all events with ste:hasRAGContext "yes" to show RAG was used
- Only extract events explicitly mentioned in the current chunk
- If no clear events are found in current chunk, return empty
"""

        try:
            response = llm.invoke([HumanMessage(content=enhanced_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            self.stats['rag_queries'] += 1  # Count as RAG usage
            logger.info(f"Generated RAG-enhanced RDF for chunk {chunk_num}")
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num} with RAG: {e}")
            return ""

    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."

        context_parts = ["\n=== Location Information ==="]

        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")

            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
            else:
                context_parts.append("  - Coordinates: Not available")

            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")

            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")

            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")

        return "\n".join(context_parts)

    def generate_global_location_rdf(self) -> str:
        """Generate RDF for all unique locations found across all chunks"""
        if not self.global_locations:
            return ""

        location_rdf_parts = []

        for location_key, location_info in self.global_locations.items():
            clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
            location_id = f"ste:Location_{clean_name}"

            rdf_lines = [f'{location_id} a ste:Location ;']
            rdf_lines.append(f'    rdfs:label "{location_info.name}" ;')

            if location_info.latitude and location_info.longitude:
                rdf_lines.append(f'    geo:lat "{location_info.latitude}"^^xsd:double ;')
                rdf_lines.append(f'    geo:long "{location_info.longitude}"^^xsd:double ;')

            if location_info.country:
                rdf_lines.append(f'    ste:hasCountry "{location_info.country}" ;')

            if location_info.region:
                rdf_lines.append(f'    ste:hasRegion "{location_info.region}" ;')

            if location_info.source:
                rdf_lines.append(f'    ste:hasSource "{location_info.source}" ;')

            if location_info.uri:
                rdf_lines.append(f'    ste:hasURI <{location_info.uri}> ;')

            if rdf_lines[-1].endswith(' ;'):
                rdf_lines[-1] = rdf_lines[-1][:-2] + ' .'

            location_rdf_parts.append('\n'.join(rdf_lines))

        return '\n\n'.join(location_rdf_parts)

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or
                stripped.startswith(':') or stripped.startswith('_') or
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

    def prepare_vectorstore(self, text_chunks: List[str]):
        """Create vector store from text chunks for RAG retrieval"""
        try:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            from langchain_community.vectorstores import FAISS

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            )

            # Prepare chunks with metadata
            documents = []
            metadatas = []

            for i, chunk in enumerate(text_chunks):
                if len(chunk.strip()) > 50:  # Only include substantial chunks
                    documents.append(chunk)
                    metadatas.append({
                        'chunk_id': i,
                        'length': len(chunk),
                        'type': 'text_chunk'
                    })

            if documents:
                self.vectorstore = FAISS.from_texts(documents, embeddings, metadatas=metadatas)
                self.document_chunks = documents
                logger.info(f"Created vector store with {len(documents)} chunks")
                return True
            else:
                logger.warning("No suitable chunks found for vector store")
                return False

        except Exception as e:
            logger.error(f"Error creating vector store: {e}")
            return False

    def rag_query(self, query: str, llm, k: int = 2) -> Dict[str, Any]:
        """Perform RAG query: retrieve relevant chunks + KG facts, then generate response"""
        self.stats['rag_queries'] += 1
        logger.info(f"Processing RAG query: '{query[:50]}...'")

        if not self.vectorstore:
            return {"error": "Vector store not initialized. Call prepare_vectorstore() first."}

        try:
            # 1. RETRIEVE: Get relevant text chunks
            relevant_docs = self.vectorstore.similarity_search(query, k=k)
            retrieved_chunks = [doc.page_content for doc in relevant_docs]

            # 2. EXTRACT: Get entities from query
            query_entities = self.extract_entities_advanced(query)

            # 3. RETRIEVE: Get KG facts for entities
            kg_facts = self.retrieve_kg_facts_enhanced(query_entities)
            kg_context = self.format_kg_context_enhanced(kg_facts)

            # 4. EXTRACT & ENRICH: Get locations from query
            query_locations = self.location_extractor.extract_locations_from_text(query)
            enriched_locations = {}
            for location_name in query_locations[:5]:
                location_info = self.location_extractor.enrich_location(location_name)
                if location_info:
                    enriched_locations[location_name] = location_info

            location_context = self.format_location_context(enriched_locations)

            # 5. AUGMENT: Create enhanced context for generation
            context_parts = [
                f"QUERY: {query}",
                f"\nRETRIEVED RELEVANT TEXT CHUNKS:",
                "\n" + "\n---\n".join(retrieved_chunks),
                f"\nKNOWLEDGE GRAPH CONTEXT:",
                kg_context,
                f"\nLOCATION CONTEXT:",
                location_context
            ]

            enhanced_context = "\n".join(context_parts)

            # 6. GENERATE: Create comprehensive response
            rag_prompt = f"""You are an expert historian with access to multiple knowledge sources. Answer the question using the provided context.

{enhanced_context}

INSTRUCTIONS:
1. Answer the question comprehensively using ALL available context
2. Cite specific information from the retrieved text chunks
3. Incorporate relevant knowledge graph facts to enhance your answer
4. Include precise location information and coordinates when relevant
5. If information conflicts between sources, mention this
6. Be specific about dates, places, people, and events
7. If you cannot answer completely, explain what information is missing

QUESTION: {query}

ANSWER:"""

            # 7. GENERATE: Get LLM response
            response = llm.invoke([HumanMessage(content=rag_prompt)])

            return {
                "query": query,
                "answer": response.content,
                "retrieved_chunks": retrieved_chunks,
                "entities_found": query_entities,
                "kg_facts_count": sum(len(facts) for facts in kg_facts.values()),
                "locations_found": list(enriched_locations.keys()),
                "sources": {
                    "text_chunks": len(retrieved_chunks),
                    "kg_sources": list(set(fact.source for facts in kg_facts.values() for fact in facts)),
                    "location_sources": list(set(loc.source for loc in enriched_locations.values()))
                }
            }

        except Exception as e:
            logger.error(f"RAG query failed: {e}")
            return {"error": str(e)}

    def interactive_rag_session(self, llm):
        """Start an interactive RAG session"""
        print("\n🤖 Starting Interactive RAG Session")
        print("Ask questions about your text. Type 'quit' to exit.")
        print("=" * 60)

        while True:
            try:
                query = input("\n❓ Your question: ").strip()

                if query.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye!")
                    break

                if not query:
                    continue

                print("\n🔍 Processing your question...")
                result = self.rag_query(query, llm)

                if "error" in result:
                    print(f"❌ Error: {result['error']}")
                    continue

                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")

            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue

# Utility functions
def load_api_key():
    """Llama via Ollama doesn't need an API key"""
    print("✅ Using local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Llama LLM"""
    try:
        llm = ChatOllama(model="llama3.2", temperature=0)
        print("✅ Llama LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        print("💡 Make sure Ollama is running: ollama serve")
        print("💡 And pull the model: ollama pull llama3.2")
        return None

def prepare_vectorstore_from_text(text: str, multi_kg_system):
    """Create vector store from text"""
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        from langchain_community.vectorstores import FAISS

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )

        # Split text into sentences for better retrieval
        sentences = re.split(r'[.!?]+', text)
        texts = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 20]

        if not texts:
            return None

        vectorstore = FAISS.from_texts(texts, embeddings)
        multi_kg_system.vectorstore = vectorstore
        multi_kg_system.document_chunks = texts
        print(f"📚 Vector store created with {len(texts)} text segments")
        return vectorstore
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def main():
    """Main function with chunking support and RAG capabilities - LLAMA VERSION"""
    print("🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)")

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    # Prepare vector store for RAG FIRST
    print("\n📚 Setting up RAG vector store...")
    vectorstore = prepare_vectorstore_from_text(domain_text, multi_kg_system)

    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    if token_count > 10000:  # Reduced for Llama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Extract events and create RDF
    all_turtle_outputs = []
    all_entities = set()

    print("\n🔄 Processing chunks for event extraction...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)

        if i < len(chunks):
            time.sleep(1)

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""

        final_output = prefixes + "# Historical Events with RAG-Enhanced Embedded Location Data (LLAMA)\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved enhanced RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        print(f"   - Location duplicates avoided: {multi_kg_system.stats['location_duplicates_avoided']}")
        print(f"   - Unique global locations: {len(multi_kg_system.global_locations)}")
        print(f"   - RAG queries for RDF generation: {multi_kg_system.stats['rag_queries']}")

        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")

        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")

        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    # START RAG SESSION HERE!
    if vectorstore:
        print(f"\n🤖 RAG System Ready!")

        # Show example queries
        print(f"\n💡 Try asking questions like:")
        print(f"   - 'What battles happened in Sicily?'")
        print(f"   - 'Who were the main leaders mentioned?'")
        print(f"   - 'What events occurred in 415 BC?'")
        print(f"   - 'Describe the naval engagements'")
        print(f"   - 'What was the outcome of the siege?'")

        # Ask if user wants to start interactive session
        response = input(f"\n❓ Start interactive RAG session? (y/n): ").strip()

        # Check if user typed a question instead of y/n
        if response.lower() not in ['y', 'yes', 'n', 'no', '']:
            # User typed a question directly!
            print(f"\n🔍 Processing your question: '{response}'")
            result = multi_kg_system.rag_query(response, llm)

            if "error" not in result:
                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")
            else:
                print(f"❌ Error: {result['error']}")

            # Ask if they want to continue with interactive session
            continue_response = input(f"\n❓ Continue with interactive RAG session? (y/n): ").strip().lower()
            if continue_response in ['y', 'yes', '']:
                multi_kg_system.interactive_rag_session(llm)

        elif response.lower() in ['y', 'yes', '']:
            multi_kg_system.interactive_rag_session(llm)
        else:
            print(f"\n💡 You can also query programmatically:")
            print(f"   result = multi_kg_system.rag_query('your question', llm)")

            # Offer a few sample queries
            sample_queries = [
                "What are the main events mentioned in the text?",
                "Which locations are mentioned?",
                "Who are the key people involved?"
            ]

            print(f"\n🔍 Running sample queries:")
            for query in sample_queries:
                print(f"\n❓ Sample query: '{query}'")
                result = multi_kg_system.rag_query(query, llm)
                if "error" not in result:
                    print(f"📝 Answer: {result['answer'][:200]}...")
                    print(f"📊 Sources: {len(result['retrieved_chunks'])} chunks, {result['kg_facts_count']} KG facts")
                else:
                    print(f"❌ Error: {result['error']}")
                print("-" * 40)

    else:
        print("⚠️  Could not create vector store for RAG functionality")

    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for RDF results.")

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Verifying Langchain Ollama import...
✅ ChatOllama successfully imported after reinstallation.
🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)
✅ Using local Ollama - no API key needed.
Loaded text from /content/drive/MyDrive/part_aa
📄 Using YOUR text from /content/drive/MyDrive/part_aa
📝 Text length: 398568 characters
✅ Llama LLM initialized successfully.

📚 Setting up RAG vector store...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📚 Vector store created with 1980 text segments
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 10 chunks

🔄 Processing chunks for event extraction...

🔄 Processing chunk 1/10...


This
This
Author
Author
Translator
Translator
Release Date
Release Date



🔄 Processing chunk 2/10...

🔄 Processing chunk 3/10...


...


🔄 Processing chunk 4/10...

🔄 Processing chunk 5/10...

🔄 Processing chunk 6/10...

🔄 Processing chunk 7/10...


Second Year
Second Year



🔄 Processing chunk 8/10...


Third Year
Third Year



🔄 Processing chunk 9/10...

🔄 Processing chunk 10/10...

✅ Saved enhanced RDF to /content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama_1.ttl
📊 Processing Statistics:
   - Total chunks processed: 10
   - Successful chunks: 10
   - Unique entities found: 97
   - Total KG facts retrieved: 4
   - Cache hits: 252
   - Locations found: 10
   - Locations with coordinates: 10
   - Location duplicates avoided: 5
   - Unique global locations: 5
   - RAG queries for RDF generation: 10

🔗 Knowledge Graph Connector Statistics:
   - Wikidata: 26/34 requests (76.5% success)
   - DBpedia: 21/29 requests (72.4% success)
   - ConceptNet: 2/45 requests (4.4% success)
   - Location enrichment: 55/178 locations enriched (30.9% success)

📝 Sample of generated RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://db

In [ ]:

# Cell 1: Mount Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# --- Dependency Compatibility Guard ---
import os

# Force compatible versions for transformers and sentence-transformers
os.system("pip install transformers==4.40.2 sentence-transformers==2.4.0 --quiet")

# (Optional) If you use torch, ensure it's compatible too:
# os.system("pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu --quiet")

# Now safe to import

!pip install  torch accelerate rdflib


"""
Enhanced Multi-Knowledge Graph RAG System with Text Chunking - LLAMA VERSION
Handles large texts by processing them in chunks to avoid token limits
"""

import os
import re
import time
import json
import numpy as np
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal # <--- This line is crucial
from rdflib.namespace import XSD, SKOS
# Add a direct import test here to diagnose immediately
print("\nVerifying Langchain Ollama import...")
try:
    from langchain_ollama import ChatOllama
    print("✅ ChatOllama successfully imported after reinstallation.")
except ImportError as e:
    print(f"❌ CRITICAL ERROR: ChatOllama still cannot be imported after reinstallation: {e}")
    print("Please check your pip installation output for errors and ensure your Colab runtime is healthy.")
    # You might want to exit here if this is a persistent issue
    # import sys
    # sys.exit(1)
except Exception as e:
    print(f"❌ An unexpected error occurred during ChatOllama import verification: {e}")
    # import sys
    # sys.exit(1)


# Configuration
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"
ONTOLOGY_PATH = "/content/drive/MyDrive/wiki.owl"
LOCATION_ONTOLOGY_PATH = "/content/drive/MyDrive/locations.owl"
OUTPUT_RAG_TTL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl'
OUTPUT_RAG_OWL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.owl'
KG_CACHE_FILE = '/content/drive/MyDrive/kg_cache.json'
LOCATION_CACHE_FILE = '/content/drive/MyDrive/location_cache.json'
KG_ANALYSIS_REPORT = '/content/drive/MyDrive/multi_kg_analysis_report.txt'

# Token limits - adjusted for Llama
MAX_TOKENS_PER_REQUEST = 50000  # Conservative limit for Llama
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_ollama import ChatOllama  # Changed from OpenAI to Ollama
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("pip install rdflib python-dotenv langchain langchain-ollama langchain-community faiss-cpu sentence-transformers requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""

    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()

    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}

    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")

    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None

    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)

        return list(set(filtered_locations))

    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None

        try:
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """

            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")

        return None

    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])

                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )

        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")

        return None

    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata with disambiguation"""
        try:
            time.sleep(0.5)

            # Try multiple query strategies to get the right location
            queries = [
                # Try exact label match first
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """,
                # Try with additional filters for places/locations
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  ?item wdt:P31/wdt:P279* wd:Q486972 .  # human settlement
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """
            ]

            for query in queries:
                params = {'query': query, 'format': 'json'}
                response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    bindings = data.get('results', {}).get('bindings', [])

                    if bindings:
                        # Prefer results with country information
                        best_binding = None
                        for binding in bindings:
                            if binding.get('country'):
                                best_binding = binding
                                break

                        if not best_binding:
                            best_binding = bindings[0]

                        coord_str = best_binding.get('coord', {}).get('value', '')

                        coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                        if coord_match:
                            longitude = float(coord_match.group(1))
                            latitude = float(coord_match.group(2))

                            return LocationInfo(
                                name=location_name,
                                latitude=latitude,
                                longitude=longitude,
                                country=best_binding.get('countryLabel', {}).get('value', ''),
                                source="wikidata",
                                uri=best_binding.get('item', {}).get('value', '')
                            )

        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")

        return None

    def validate_coordinates(self, location_info: LocationInfo) -> bool:
        """Validate that coordinates make sense for the location"""
        if not location_info.latitude or not location_info.longitude:
            return True

        lat, lon = location_info.latitude, location_info.longitude

        # Basic coordinate range validation
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            logger.warning(f"Invalid coordinates for {location_info.name}: {lat}, {lon}")
            return False

        return True

    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None

        location_info = None

        location_info = self.get_location_from_ontology(location_name)

        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)

        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)

        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None

        self._save_location_cache()

        if location_info:
            self.validate_coordinates(location_info)

        return location_info

class TextChunker:
    """Handles text chunking to manage token limits for Llama"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count for Llama (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 10000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""

    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0

    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""

    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata with timeout protection"""
        try:
            self._rate_limit_wait()

            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}

              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)

              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
            else:
                logger.warning(f"Wikidata returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"Wikidata query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"Wikidata query failed for '{entity}': {e}")

        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""

    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia with timeout protection"""
        try:
            self._rate_limit_wait()

            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')

                    predicate_name = predicate.split('/')[-1].replace('_', ' ')

                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
            else:
                logger.warning(f"DBpedia returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"DBpedia query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"DBpedia query failed for '{entity}': {e}")

        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector with dynamic concept discovery"""

    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)

    def search_related_concepts(self, entity: str) -> List[str]:
        """Search for related concepts using ConceptNet's search API"""
        try:
            # Try search API first
            search_url = f"{self.base_url}/search?text={entity.replace(' ', '%20')}&limit=10"
            response = requests.get(search_url, timeout=10)

            related_concepts = []
            if response.status_code == 200:
                data = response.json()
                for edge in data.get('edges', []):
                    start = edge.get('start', {}).get('label', '')
                    end = edge.get('end', {}).get('label', '')

                    # Extract concept paths and clean them
                    for concept_path in [start, end]:
                        if concept_path and '/c/en/' in concept_path:
                            concept = concept_path.replace('/c/en/', '').replace('_', ' ')
                            if concept.lower() != entity.lower() and len(concept) > 2:
                                related_concepts.append(concept)

            return list(set(related_concepts))[:5]  # Return top 5 unique concepts

        except Exception as e:
            logger.debug(f"ConceptNet search failed for {entity}: {e}")
            return []

    def query_concept_directly(self, concept: str, limit: int = 20) -> List[dict]:
        """Query a specific concept and return raw edges"""
        try:
            concept_path = f"/c/en/{concept.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept_path}?limit={limit}"

            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data.get('edges', [])

        except Exception as e:
            logger.debug(f"ConceptNet direct query failed for {concept}: {e}")

        return []

    def retrieve_facts(self, entity: str, limit: int = 100) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet through dynamic discovery"""
        try:
            self._rate_limit_wait()
            all_facts = []

            # Strategy 1: Try direct query first
            direct_edges = self.query_concept_directly(entity, limit//2)

            # Strategy 2: Search for related concepts and query them
            related_concepts = self.search_related_concepts(entity)

            # Process direct edges
            for edge in direct_edges:
                fact = self._edge_to_fact(edge, entity, "direct")
                if fact:
                    all_facts.append(fact)

            # Process related concept edges
            for concept in related_concepts:
                concept_edges = self.query_concept_directly(concept, 5)
                for edge in concept_edges:
                    fact = self._edge_to_fact(edge, entity, f"via_{concept}")
                    if fact:
                        all_facts.append(fact)

            if all_facts:
                self.success_count += 1
                logger.info(f"Retrieved {len(all_facts)} facts from ConceptNet for '{entity}'")
                if related_concepts:
                    logger.info(f"  - Found related concepts: {related_concepts}")

            return all_facts[:limit]

        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")

        return []

    def _edge_to_fact(self, edge: dict, original_entity: str, discovery_method: str) -> Optional[EnhancedKnowledgeFact]:
        """Convert ConceptNet edge to EnhancedKnowledgeFact"""
        try:
            start = edge.get('start', {})
            end = edge.get('end', {})
            relation = edge.get('rel', {})
            weight = edge.get('weight', 1.0)

            start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
            end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
            rel_label = relation.get('label', 'related_to')

            # Skip if labels are empty or too short
            if not start_label or not end_label or len(start_label) < 2 or len(end_label) < 2:
                return None

            # Determine confidence based on discovery method
            confidence_multiplier = 1.0 if discovery_method == "direct" else 0.6

            return EnhancedKnowledgeFact(
                subject=original_entity,
                predicate=rel_label,
                object=end_label if start_label.lower() in original_entity.lower() else start_label,
                source=self.name,
                confidence=min(weight * confidence_multiplier, 1.0),
                context=f"Discovered {discovery_method}"
            )

        except Exception as e:
            logger.debug(f"Error converting edge to fact: {e}")
            return None

class MultiKGCache:
    """Caching system for knowledge graph facts"""

    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}

    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")

    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"

    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)

    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph RAG system with chunking and location extraction - LLAMA VERSION"""

    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.global_locations = {}
        self.vectorstore = None  # For RAG retrieval
        self.document_chunks = []  # Store processed chunks for RAG
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0,
            'location_duplicates_avoided': 0,
            'rag_queries': 0
        }

    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []

        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:25]

    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs with improved timeout handling"""
        all_facts = {}
        cache_hits = 0

        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}

            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 10)
                        futures[future] = (entity, source_name)

            # Collect results with better timeout handling
            completed = 0
            total_futures = len(futures)

            try:
                for future in as_completed(futures, timeout=45):  # Increased timeout
                    entity, source_name = futures[future]
                    completed += 1

                    try:
                        facts = future.result(timeout=5)  # Individual future timeout
                        if facts:
                            self.cache.set(source_name, entity, facts)

                            if entity not in all_facts:
                                all_facts[entity] = []
                            all_facts[entity].extend(facts)

                            self.stats['facts_retrieved'] += len(facts)

                        logger.debug(f"✅ {source_name} completed for {entity} ({completed}/{total_futures})")

                    except Exception as e:
                        logger.warning(f"❌ {source_name} failed for {entity}: {e}")
                        continue

            except TimeoutError:
                pending_count = total_futures - completed
                logger.warning(f"⏰ Timeout: {pending_count}/{total_futures} KG queries still pending, continuing with available results")

                # Cancel remaining futures
                for future in futures:
                    if not future.done():
                        future.cancel()

        self.stats['cache_hits'] += cache_hits
        logger.info(f"KG retrieval completed: {completed}/{total_futures} successful, {cache_hits} cache hits")
        return all_facts

    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []

        for entity, facts in kg_facts.items():
            if facts:
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)

                context_parts.append(f"\n=== Knowledge about {entity} ===")

                by_source = {}
                for fact in sorted_facts[:8]:
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)

                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:4]:
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)

        return "\n".join(context_parts)

    def register_global_location(self, location_info: LocationInfo) -> str:
        """Register location globally and return unique identifier"""
        location_key = location_info.name.lower().strip()

        if location_key in self.global_locations:
            existing = self.global_locations[location_key]
            if (location_info.latitude and location_info.longitude and
                (not existing.latitude or not existing.longitude)):
                self.global_locations[location_key] = location_info
                logger.info(f"Updated coordinates for {location_info.name}")
            else:
                self.stats['location_duplicates_avoided'] += 1
                logger.debug(f"Location {location_info.name} already registered")
        else:
            self.global_locations[location_key] = location_info
            logger.info(f"Registered new location: {location_info.name}")

        clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
        return f"ste:Location_{clean_name}"

    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text with RAG-enhanced location extraction"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars)")

        # 1. RAG RETRIEVAL: Get relevant context from vector store
        relevant_context = ""
        if self.vectorstore:
            try:
                # Use the chunk as a query to retrieve similar/relevant text
                relevant_docs = self.vectorstore.similarity_search(chunk, k=15)
                retrieved_chunks = [doc.page_content for doc in relevant_docs]
                relevant_context = "\n---\n".join(retrieved_chunks)
                logger.info(f"Retrieved {len(retrieved_chunks)} relevant chunks via RAG for chunk {chunk_num}")
            except Exception as e:
                logger.warning(f"RAG retrieval failed for chunk {chunk_num}: {e}")
                relevant_context = ""

        # 2. Extract entities and locations
        entities = self.extract_entities_advanced(chunk)
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")

        # 3. Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:15]:  # Limit to 5 to avoid overwhelming
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                self.register_global_location(location_info)
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1

        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")

        # 4. Get KG facts for entities
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        location_context = self.format_location_context(enriched_locations)

        # 5. RAG-ENHANCED PROMPT: Use retrieved context + KG facts + locations
        enhanced_prompt = f"""You are extracting historical events using RAG (Retrieval-Augmented Generation). Use ALL available context sources to enhance your extraction.

CURRENT TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

RAG RETRIEVED RELEVANT CONTEXT:
{relevant_context if relevant_context else "No relevant context retrieved."}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in the current text chunk. Use the RAG retrieved context, KG facts, and location coordinates to enhance details but stay faithful to what's actually in the current chunk.

Requirements:
1. Extract ONLY events mentioned in the CURRENT text chunk (not from retrieved context)
2. Use RAG retrieved context to provide additional historical context and validation
3. Use KG facts to enhance entity information
4. Use location coordinates to provide precise geographical data
5. Include ALL these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasRegion (region if available)
   - ste:hasLocationSource (source of coordinates: wikidata/dbpedia/local_ontology)
   - ste:hasResult (outcome/consequence)
   - ste:hasRAGContext "yes" (to indicate this was RAG-enhanced)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from current chunk" ;
    ste:hasAgent "specific person from current chunk" ;
    ste:hasTime "specific date from current chunk" ;
    ste:hasLocation "specific location from current chunk" ;
    ste:hasLatitude "37.1234"^^xsd:double ;
    ste:hasLongitude "15.5678"^^xsd:double ;
    ste:hasCountry "Italy" ;
    ste:hasRegion "Sicily" ;
    ste:hasLocationSource "wikidata" ;
    ste:hasResult "specific outcome from current chunk" ;
    ste:hasRAGContext "yes" .
```

IMPORTANT:
- The PRIMARY source is the CURRENT text chunk - extract events from IT
- Use RAG retrieved context to validate and enhance your understanding
- Use KG facts to enrich entity details
- Include precise coordinates from location sources
- Mark all events with ste:hasRAGContext "yes" to show RAG was used
- Only extract events explicitly mentioned in the current chunk
- If no clear events are found in current chunk, return empty
"""

        try:
            response = llm.invoke([HumanMessage(content=enhanced_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            self.stats['rag_queries'] += 1  # Count as RAG usage
            logger.info(f"Generated RAG-enhanced RDF for chunk {chunk_num}")
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num} with RAG: {e}")
            return ""

    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."

        context_parts = ["\n=== Location Information ==="]

        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")

            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
            else:
                context_parts.append("  - Coordinates: Not available")

            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")

            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")

            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")

        return "\n".join(context_parts)

    def generate_global_location_rdf(self) -> str:
        """Generate RDF for all unique locations found across all chunks"""
        if not self.global_locations:
            return ""

        location_rdf_parts = []

        for location_key, location_info in self.global_locations.items():
            clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
            location_id = f"ste:Location_{clean_name}"

            rdf_lines = [f'{location_id} a ste:Location ;']
            rdf_lines.append(f'    rdfs:label "{location_info.name}" ;')

            if location_info.latitude and location_info.longitude:
                rdf_lines.append(f'    geo:lat "{location_info.latitude}"^^xsd:double ;')
                rdf_lines.append(f'    geo:long "{location_info.longitude}"^^xsd:double ;')

            if location_info.country:
                rdf_lines.append(f'    ste:hasCountry "{location_info.country}" ;')

            if location_info.region:
                rdf_lines.append(f'    ste:hasRegion "{location_info.region}" ;')

            if location_info.source:
                rdf_lines.append(f'    ste:hasSource "{location_info.source}" ;')

            if location_info.uri:
                rdf_lines.append(f'    ste:hasURI <{location_info.uri}> ;')

            if rdf_lines[-1].endswith(' ;'):
                rdf_lines[-1] = rdf_lines[-1][:-2] + ' .'

            location_rdf_parts.append('\n'.join(rdf_lines))

        return '\n\n'.join(location_rdf_parts)

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or
                stripped.startswith(':') or stripped.startswith('_') or
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

    def prepare_vectorstore(self, text_chunks: List[str]):
        """Create vector store from text chunks for RAG retrieval"""
        try:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            from langchain_community.vectorstores import FAISS

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            )

            # Prepare chunks with metadata
            documents = []
            metadatas = []

            for i, chunk in enumerate(text_chunks):
                if len(chunk.strip()) > 50:  # Only include substantial chunks
                    documents.append(chunk)
                    metadatas.append({
                        'chunk_id': i,
                        'length': len(chunk),
                        'type': 'text_chunk'
                    })

            if documents:
                self.vectorstore = FAISS.from_texts(documents, embeddings, metadatas=metadatas)
                self.document_chunks = documents
                logger.info(f"Created vector store with {len(documents)} chunks")
                return True
            else:
                logger.warning("No suitable chunks found for vector store")
                return False

        except Exception as e:
            logger.error(f"Error creating vector store: {e}")
            return False

    def rag_query(self, query: str, llm, k: int = 3) -> Dict[str, Any]:
        """Perform RAG query: retrieve relevant chunks + KG facts, then generate response"""
        self.stats['rag_queries'] += 1
        logger.info(f"Processing RAG query: '{query[:50]}...'")

        if not self.vectorstore:
            return {"error": "Vector store not initialized. Call prepare_vectorstore() first."}

        try:
            # 1. RETRIEVE: Get relevant text chunks
            relevant_docs = self.vectorstore.similarity_search(query, k=k)
            retrieved_chunks = [doc.page_content for doc in relevant_docs]

            # 2. EXTRACT: Get entities from query
            query_entities = self.extract_entities_advanced(query)

            # 3. RETRIEVE: Get KG facts for entities
            kg_facts = self.retrieve_kg_facts_enhanced(query_entities)
            kg_context = self.format_kg_context_enhanced(kg_facts)

            # 4. EXTRACT & ENRICH: Get locations from query
            query_locations = self.location_extractor.extract_locations_from_text(query)
            enriched_locations = {}
            for location_name in query_locations[:5]:
                location_info = self.location_extractor.enrich_location(location_name)
                if location_info:
                    enriched_locations[location_name] = location_info

            location_context = self.format_location_context(enriched_locations)

            # 5. AUGMENT: Create enhanced context for generation
            context_parts = [
                f"QUERY: {query}",
                f"\nRETRIEVED RELEVANT TEXT CHUNKS:",
                "\n" + "\n---\n".join(retrieved_chunks),
                f"\nKNOWLEDGE GRAPH CONTEXT:",
                kg_context,
                f"\nLOCATION CONTEXT:",
                location_context
            ]

            enhanced_context = "\n".join(context_parts)

            # 6. GENERATE: Create comprehensive response
            rag_prompt = f"""You are an expert historian with access to multiple knowledge sources. Answer the question using the provided context.

{enhanced_context}

INSTRUCTIONS:
1. Answer the question comprehensively using ALL available context
2. Cite specific information from the retrieved text chunks
3. Incorporate relevant knowledge graph facts to enhance your answer
4. Include precise location information and coordinates when relevant
5. If information conflicts between sources, mention this
6. Be specific about dates, places, people, and events
7. If you cannot answer completely, explain what information is missing

QUESTION: {query}

ANSWER:"""

            # 7. GENERATE: Get LLM response
            response = llm.invoke([HumanMessage(content=rag_prompt)])

            return {
                "query": query,
                "answer": response.content,
                "retrieved_chunks": retrieved_chunks,
                "entities_found": query_entities,
                "kg_facts_count": sum(len(facts) for facts in kg_facts.values()),
                "locations_found": list(enriched_locations.keys()),
                "sources": {
                    "text_chunks": len(retrieved_chunks),
                    "kg_sources": list(set(fact.source for facts in kg_facts.values() for fact in facts)),
                    "location_sources": list(set(loc.source for loc in enriched_locations.values()))
                }
            }

        except Exception as e:
            logger.error(f"RAG query failed: {e}")
            return {"error": str(e)}

    def interactive_rag_session(self, llm):
        """Start an interactive RAG session"""
        print("\n🤖 Starting Interactive RAG Session")
        print("Ask questions about your text. Type 'quit' to exit.")
        print("=" * 60)

        while True:
            try:
                query = input("\n❓ Your question: ").strip()

                if query.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye!")
                    break

                if not query:
                    continue

                print("\n🔍 Processing your question...")
                result = self.rag_query(query, llm)

                if "error" in result:
                    print(f"❌ Error: {result['error']}")
                    continue

                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")

            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue

# Utility functions
def load_api_key():
    """Llama via Ollama doesn't need an API key"""
    print("✅ Using local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Llama LLM"""
    try:
        llm = ChatOllama(model="llama3.2", temperature=0)
        print("✅ Llama LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        print("💡 Make sure Ollama is running: ollama serve")
        print("💡 And pull the model: ollama pull llama3.2")
        return None

def prepare_vectorstore_from_text(text: str, multi_kg_system):
    """Create vector store from text"""
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        from langchain_community.vectorstores import FAISS

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )

        # Split text into sentences for better retrieval
        sentences = re.split(r'[.!?]+', text)
        texts = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 20]

        if not texts:
            return None

        vectorstore = FAISS.from_texts(texts, embeddings)
        multi_kg_system.vectorstore = vectorstore
        multi_kg_system.document_chunks = texts
        print(f"📚 Vector store created with {len(texts)} text segments")
        return vectorstore
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def main():
    """Main function with chunking support and RAG capabilities - LLAMA VERSION"""
    print("🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)")

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    # Prepare vector store for RAG FIRST
    print("\n📚 Setting up RAG vector store...")
    vectorstore = prepare_vectorstore_from_text(domain_text, multi_kg_system)

    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    if token_count > 10000:  # Reduced for Llama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Extract events and create RDF
    all_turtle_outputs = []
    all_entities = set()

    print("\n🔄 Processing chunks for event extraction...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)

        if i < len(chunks):
            time.sleep(1)

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""

        final_output = prefixes + "# Historical Events with RAG-Enhanced Embedded Location Data (LLAMA)\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved enhanced RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        print(f"   - Location duplicates avoided: {multi_kg_system.stats['location_duplicates_avoided']}")
        print(f"   - Unique global locations: {len(multi_kg_system.global_locations)}")
        print(f"   - RAG queries for RDF generation: {multi_kg_system.stats['rag_queries']}")

        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")

        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")

        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    # START RAG SESSION HERE!
    if vectorstore:
        print(f"\n🤖 RAG System Ready!")

        # Show example queries
        print(f"\n💡 Try asking questions like:")
        print(f"   - 'What battles happened in Sicily?'")
        print(f"   - 'Who were the main leaders mentioned?'")
        print(f"   - 'What events occurred in 415 BC?'")
        print(f"   - 'Describe the naval engagements'")
        print(f"   - 'What was the outcome of the siege?'")

        # Ask if user wants to start interactive session
        response = input(f"\n❓ Start interactive RAG session? (y/n): ").strip()

        # Check if user typed a question instead of y/n
        if response.lower() not in ['y', 'yes', 'n', 'no', '']:
            # User typed a question directly!
            print(f"\n🔍 Processing your question: '{response}'")
            result = multi_kg_system.rag_query(response, llm)

            if "error" not in result:
                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")
            else:
                print(f"❌ Error: {result['error']}")

            # Ask if they want to continue with interactive session
            continue_response = input(f"\n❓ Continue with interactive RAG session? (y/n): ").strip().lower()
            if continue_response in ['y', 'yes', '']:
                multi_kg_system.interactive_rag_session(llm)

        elif response.lower() in ['y', 'yes', '']:
            multi_kg_system.interactive_rag_session(llm)
        else:
            print(f"\n💡 You can also query programmatically:")
            print(f"   result = multi_kg_system.rag_query('your question', llm)")

            # Offer a few sample queries
            sample_queries = [
                "What are the main events mentioned in the text?",
                "Which locations are mentioned?",
                "Who are the key people involved?"
            ]

            print(f"\n🔍 Running sample queries:")
            for query in sample_queries:
                print(f"\n❓ Sample query: '{query}'")
                result = multi_kg_system.rag_query(query, llm)
                if "error" not in result:
                    print(f"📝 Answer: {result['answer'][:200]}...")
                    print(f"📊 Sources: {len(result['retrieved_chunks'])} chunks, {result['kg_facts_count']} KG facts")
                else:
                    print(f"❌ Error: {result['error']}")
                print("-" * 40)

    else:
        print("⚠️  Could not create vector store for RAG functionality")

    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for RDF results.")

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Verifying Langchain Ollama import...
✅ ChatOllama successfully imported after reinstallation.
🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)
✅ Using local Ollama - no API key needed.
Loaded text from /content/drive/MyDrive/part_aa
📄 Using YOUR text from /content/drive/MyDrive/part_aa
📝 Text length: 398568 characters
✅ Llama LLM initialized successfully.

📚 Setting up RAG vector store...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📚 Vector store created with 1980 text segments
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 10 chunks

🔄 Processing chunks for event extraction...

🔄 Processing chunk 1/10...


This
This
Author
Author
Translator
Translator
Release Date
Release Date
Character
Character



🔄 Processing chunk 2/10...


...


🔄 Processing chunk 8/10...


Third Year
Third Year

The

The



🔄 Processing chunk 9/10...



🔄 Processing chunk 10/10...



✅ Saved enhanced RDF to /content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl
📊 Processing Statistics:
   - Total chunks processed: 10
   - Successful chunks: 10
   - Unique entities found: 183
   - Total KG facts retrieved: 528
   - Cache hits: 402
   - Locations found: 61
   - Locations with coordinates: 61
   - Location duplicates avoided: 34
   - Unique global locations: 27
   - RAG queries for RDF generation: 10

🔗 Knowledge Graph Connector Statistics:
   - Wikidata: 70/80 requests (87.5% success)
   - DBpedia: 63/73 requests (86.3% success)
   - ConceptNet: 15/91 requests (16.5% success)
   - Location enrichment: 69/228 locations enriched (30.3% success)

📝 Sample of generated RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.o

In [ ]:

# Cell 1: Mount Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# --- Dependency Compatibility Guard ---
import os

# Force compatible versions for transformers and sentence-transformers
os.system("pip install transformers==4.40.2 sentence-transformers==2.4.0 --quiet")

# (Optional) If you use torch, ensure it's compatible too:
# os.system("pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu --quiet")

# Now safe to import

!pip install  torch accelerate rdflib


"""
Enhanced Multi-Knowledge Graph RAG System with Text Chunking - LLAMA VERSION
Handles large texts by processing them in chunks to avoid token limits
"""

import os
import re
import time
import json
import numpy as np
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal # <--- This line is crucial
from rdflib.namespace import XSD, SKOS
# Add a direct import test here to diagnose immediately
print("\nVerifying Langchain Ollama import...")
try:
    from langchain_ollama import ChatOllama
    print("✅ ChatOllama successfully imported after reinstallation.")
except ImportError as e:
    print(f"❌ CRITICAL ERROR: ChatOllama still cannot be imported after reinstallation: {e}")
    print("Please check your pip installation output for errors and ensure your Colab runtime is healthy.")
    # You might want to exit here if this is a persistent issue
    # import sys
    # sys.exit(1)
except Exception as e:
    print(f"❌ An unexpected error occurred during ChatOllama import verification: {e}")
    # import sys
    # sys.exit(1)


# Configuration
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"
ONTOLOGY_PATH = "/content/drive/MyDrive/wiki.owl"
LOCATION_ONTOLOGY_PATH = "/content/drive/MyDrive/locations.owl"
OUTPUT_RAG_TTL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl'
OUTPUT_RAG_OWL = '/content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.owl'
KG_CACHE_FILE = '/content/drive/MyDrive/kg_cache.json'
LOCATION_CACHE_FILE = '/content/drive/MyDrive/location_cache.json'
KG_ANALYSIS_REPORT = '/content/drive/MyDrive/multi_kg_analysis_report.txt'

# Token limits - adjusted for Llama
MAX_TOKENS_PER_REQUEST = 50000  # Conservative limit for Llama
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS
    from langchain_ollama import ChatOllama  # Changed from OpenAI to Ollama
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("pip install rdflib python-dotenv langchain langchain-ollama langchain-community faiss-cpu sentence-transformers requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""

    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()

    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}

    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")

    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None

    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)

        return list(set(filtered_locations))

    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None

        try:
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """

            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")

        return None

    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])

                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )

        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")

        return None

    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata with disambiguation"""
        try:
            time.sleep(0.5)

            # Try multiple query strategies to get the right location
            queries = [
                # Try exact label match first
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """,
                # Try with additional filters for places/locations
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  ?item wdt:P31/wdt:P279* wd:Q486972 .  # human settlement
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """
            ]

            for query in queries:
                params = {'query': query, 'format': 'json'}
                response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    bindings = data.get('results', {}).get('bindings', [])

                    if bindings:
                        # Prefer results with country information
                        best_binding = None
                        for binding in bindings:
                            if binding.get('country'):
                                best_binding = binding
                                break

                        if not best_binding:
                            best_binding = bindings[0]

                        coord_str = best_binding.get('coord', {}).get('value', '')

                        coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                        if coord_match:
                            longitude = float(coord_match.group(1))
                            latitude = float(coord_match.group(2))

                            return LocationInfo(
                                name=location_name,
                                latitude=latitude,
                                longitude=longitude,
                                country=best_binding.get('countryLabel', {}).get('value', ''),
                                source="wikidata",
                                uri=best_binding.get('item', {}).get('value', '')
                            )

        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")

        return None

    def validate_coordinates(self, location_info: LocationInfo) -> bool:
        """Validate that coordinates make sense for the location"""
        if not location_info.latitude or not location_info.longitude:
            return True

        lat, lon = location_info.latitude, location_info.longitude

        # Basic coordinate range validation
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            logger.warning(f"Invalid coordinates for {location_info.name}: {lat}, {lon}")
            return False

        return True

    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None

        location_info = None

        location_info = self.get_location_from_ontology(location_name)

        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)

        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)

        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None

        self._save_location_cache()

        if location_info:
            self.validate_coordinates(location_info)

        return location_info

class TextChunker:
    """Handles text chunking to manage token limits for Llama"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count for Llama (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 10000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""

    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0

    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""

    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata with timeout protection"""
        try:
            self._rate_limit_wait()

            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}

              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)

              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
            else:
                logger.warning(f"Wikidata returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"Wikidata query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"Wikidata query failed for '{entity}': {e}")

        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""

    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia with timeout protection"""
        try:
            self._rate_limit_wait()

            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)  # Reduced timeout

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')

                    predicate_name = predicate.split('/')[-1].replace('_', ' ')

                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
            else:
                logger.warning(f"DBpedia returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"DBpedia query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"DBpedia query failed for '{entity}': {e}")

        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector with dynamic concept discovery"""

    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)

    def search_related_concepts(self, entity: str) -> List[str]:
        """Search for related concepts using ConceptNet's search API"""
        try:
            # Try search API first
            search_url = f"{self.base_url}/search?text={entity.replace(' ', '%20')}&limit=10"
            response = requests.get(search_url, timeout=10)

            related_concepts = []
            if response.status_code == 200:
                data = response.json()
                for edge in data.get('edges', []):
                    start = edge.get('start', {}).get('label', '')
                    end = edge.get('end', {}).get('label', '')

                    # Extract concept paths and clean them
                    for concept_path in [start, end]:
                        if concept_path and '/c/en/' in concept_path:
                            concept = concept_path.replace('/c/en/', '').replace('_', ' ')
                            if concept.lower() != entity.lower() and len(concept) > 2:
                                related_concepts.append(concept)

            return list(set(related_concepts))[:5]  # Return top 5 unique concepts

        except Exception as e:
            logger.debug(f"ConceptNet search failed for {entity}: {e}")
            return []

    def query_concept_directly(self, concept: str, limit: int = 20) -> List[dict]:
        """Query a specific concept and return raw edges"""
        try:
            concept_path = f"/c/en/{concept.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept_path}?limit={limit}"

            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data.get('edges', [])

        except Exception as e:
            logger.debug(f"ConceptNet direct query failed for {concept}: {e}")

        return []

    def retrieve_facts(self, entity: str, limit: int = 100) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet through dynamic discovery"""
        try:
            self._rate_limit_wait()
            all_facts = []

            # Strategy 1: Try direct query first
            direct_edges = self.query_concept_directly(entity, limit//2)

            # Strategy 2: Search for related concepts and query them
            related_concepts = self.search_related_concepts(entity)

            # Process direct edges
            for edge in direct_edges:
                fact = self._edge_to_fact(edge, entity, "direct")
                if fact:
                    all_facts.append(fact)

            # Process related concept edges
            for concept in related_concepts:
                concept_edges = self.query_concept_directly(concept, 5)
                for edge in concept_edges:
                    fact = self._edge_to_fact(edge, entity, f"via_{concept}")
                    if fact:
                        all_facts.append(fact)

            if all_facts:
                self.success_count += 1
                logger.info(f"Retrieved {len(all_facts)} facts from ConceptNet for '{entity}'")
                if related_concepts:
                    logger.info(f"  - Found related concepts: {related_concepts}")

            return all_facts[:limit]

        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")

        return []

    def _edge_to_fact(self, edge: dict, original_entity: str, discovery_method: str) -> Optional[EnhancedKnowledgeFact]:
        """Convert ConceptNet edge to EnhancedKnowledgeFact"""
        try:
            start = edge.get('start', {})
            end = edge.get('end', {})
            relation = edge.get('rel', {})
            weight = edge.get('weight', 1.0)

            start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
            end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
            rel_label = relation.get('label', 'related_to')

            # Skip if labels are empty or too short
            if not start_label or not end_label or len(start_label) < 2 or len(end_label) < 2:
                return None

            # Determine confidence based on discovery method
            confidence_multiplier = 1.0 if discovery_method == "direct" else 0.6

            return EnhancedKnowledgeFact(
                subject=original_entity,
                predicate=rel_label,
                object=end_label if start_label.lower() in original_entity.lower() else start_label,
                source=self.name,
                confidence=min(weight * confidence_multiplier, 1.0),
                context=f"Discovered {discovery_method}"
            )

        except Exception as e:
            logger.debug(f"Error converting edge to fact: {e}")
            return None

class MultiKGCache:
    """Caching system for knowledge graph facts"""

    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}

    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")

    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"

    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)

    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph RAG system with chunking and location extraction - LLAMA VERSION"""

    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.global_locations = {}
        self.vectorstore = None  # For RAG retrieval
        self.document_chunks = []  # Store processed chunks for RAG
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0,
            'location_duplicates_avoided': 0,
            'rag_queries': 0
        }

    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []

        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:40]

    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs with improved timeout handling"""
        all_facts = {}
        cache_hits = 0

        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}

            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 15)
                        futures[future] = (entity, source_name)

            # Collect results with better timeout handling
            completed = 0
            total_futures = len(futures)

            try:
                for future in as_completed(futures, timeout=45):  # Increased timeout
                    entity, source_name = futures[future]
                    completed += 1

                    try:
                        facts = future.result(timeout=5)  # Individual future timeout
                        if facts:
                            self.cache.set(source_name, entity, facts)

                            if entity not in all_facts:
                                all_facts[entity] = []
                            all_facts[entity].extend(facts)

                            self.stats['facts_retrieved'] += len(facts)

                        logger.debug(f"✅ {source_name} completed for {entity} ({completed}/{total_futures})")

                    except Exception as e:
                        logger.warning(f"❌ {source_name} failed for {entity}: {e}")
                        continue

            except TimeoutError:
                pending_count = total_futures - completed
                logger.warning(f"⏰ Timeout: {pending_count}/{total_futures} KG queries still pending, continuing with available results")

                # Cancel remaining futures
                for future in futures:
                    if not future.done():
                        future.cancel()

        self.stats['cache_hits'] += cache_hits
        logger.info(f"KG retrieval completed: {completed}/{total_futures} successful, {cache_hits} cache hits")
        return all_facts

    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []

        for entity, facts in kg_facts.items():
            if facts:
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)

                context_parts.append(f"\n=== Knowledge about {entity} ===")

                by_source = {}
                for fact in sorted_facts[:12]:
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)

                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:8]:
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)

        return "\n".join(context_parts)

    def register_global_location(self, location_info: LocationInfo) -> str:
        """Register location globally and return unique identifier"""
        location_key = location_info.name.lower().strip()

        if location_key in self.global_locations:
            existing = self.global_locations[location_key]
            if (location_info.latitude and location_info.longitude and
                (not existing.latitude or not existing.longitude)):
                self.global_locations[location_key] = location_info
                logger.info(f"Updated coordinates for {location_info.name}")
            else:
                self.stats['location_duplicates_avoided'] += 1
                logger.debug(f"Location {location_info.name} already registered")
        else:
            self.global_locations[location_key] = location_info
            logger.info(f"Registered new location: {location_info.name}")

        clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
        return f"ste:Location_{clean_name}"

    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text with RAG-enhanced location extraction"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars)")

        # 1. RAG RETRIEVAL: Get relevant context from vector store
        relevant_context = ""
        if self.vectorstore:
            try:
                # Use the chunk as a query to retrieve similar/relevant text
                relevant_docs = self.vectorstore.similarity_search(chunk, k=25)
                retrieved_chunks = [doc.page_content for doc in relevant_docs]
                relevant_context = "\n---\n".join(retrieved_chunks)
                logger.info(f"Retrieved {len(retrieved_chunks)} relevant chunks via RAG for chunk {chunk_num}")
            except Exception as e:
                logger.warning(f"RAG retrieval failed for chunk {chunk_num}: {e}")
                relevant_context = ""

        # 2. Extract entities and locations
        entities = self.extract_entities_advanced(chunk)
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")

        # 3. Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:25]:  # Limit to 5 to avoid overwhelming
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                self.register_global_location(location_info)
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1

        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")

        # 4. Get KG facts for entities
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        location_context = self.format_location_context(enriched_locations)

        # 5. RAG-ENHANCED PROMPT: Use retrieved context + KG facts + locations
        enhanced_prompt = f"""You are extracting historical events using RAG (Retrieval-Augmented Generation). Use ALL available context sources to enhance your extraction.

CURRENT TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

RAG RETRIEVED RELEVANT CONTEXT:
{relevant_context if relevant_context else "No relevant context retrieved."}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in the current text chunk. Use the RAG retrieved context, KG facts, and location coordinates to enhance details but stay faithful to what's actually in the current chunk.

Requirements:
1. Extract ONLY events mentioned in the CURRENT text chunk (not from retrieved context)
2. Use RAG retrieved context to provide additional historical context and validation
3. Use KG facts to enhance entity information
4. Use location coordinates to provide precise geographical data
5. Include ALL these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasRegion (region if available)
   - ste:hasLocationSource (source of coordinates: wikidata/dbpedia/local_ontology)
   - ste:hasResult (outcome/consequence)
   - ste:hasRAGContext "yes" (to indicate this was RAG-enhanced)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from current chunk" ;
    ste:hasAgent "specific person from current chunk" ;
    ste:hasTime "specific date from current chunk" ;
    ste:hasLocation "specific location from current chunk" ;
    ste:hasLatitude "37.1234"^^xsd:double ;
    ste:hasLongitude "15.5678"^^xsd:double ;
    ste:hasCountry "Italy" ;
    ste:hasRegion "Sicily" ;
    ste:hasLocationSource "wikidata" ;
    ste:hasResult "specific outcome from current chunk" ;
    ste:hasRAGContext "yes" .
```

IMPORTANT:
- The PRIMARY source is the CURRENT text chunk - extract events from IT
- Use RAG retrieved context to validate and enhance your understanding
- Use KG facts to enrich entity details
- Include precise coordinates from location sources
- Mark all events with ste:hasRAGContext "yes" to show RAG was used
- Only extract events explicitly mentioned in the current chunk
- If no clear events are found in current chunk, return empty
"""

        try:
            response = llm.invoke([HumanMessage(content=enhanced_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            self.stats['rag_queries'] += 1  # Count as RAG usage
            logger.info(f"Generated RAG-enhanced RDF for chunk {chunk_num}")
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num} with RAG: {e}")
            return ""

    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."

        context_parts = ["\n=== Location Information ==="]

        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")

            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
            else:
                context_parts.append("  - Coordinates: Not available")

            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")

            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")

            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")

        return "\n".join(context_parts)

    def generate_global_location_rdf(self) -> str:
        """Generate RDF for all unique locations found across all chunks"""
        if not self.global_locations:
            return ""

        location_rdf_parts = []

        for location_key, location_info in self.global_locations.items():
            clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
            location_id = f"ste:Location_{clean_name}"

            rdf_lines = [f'{location_id} a ste:Location ;']
            rdf_lines.append(f'    rdfs:label "{location_info.name}" ;')

            if location_info.latitude and location_info.longitude:
                rdf_lines.append(f'    geo:lat "{location_info.latitude}"^^xsd:double ;')
                rdf_lines.append(f'    geo:long "{location_info.longitude}"^^xsd:double ;')

            if location_info.country:
                rdf_lines.append(f'    ste:hasCountry "{location_info.country}" ;')

            if location_info.region:
                rdf_lines.append(f'    ste:hasRegion "{location_info.region}" ;')

            if location_info.source:
                rdf_lines.append(f'    ste:hasSource "{location_info.source}" ;')

            if location_info.uri:
                rdf_lines.append(f'    ste:hasURI <{location_info.uri}> ;')

            if rdf_lines[-1].endswith(' ;'):
                rdf_lines[-1] = rdf_lines[-1][:-2] + ' .'

            location_rdf_parts.append('\n'.join(rdf_lines))

        return '\n\n'.join(location_rdf_parts)

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or
                stripped.startswith(':') or stripped.startswith('_') or
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

    def prepare_vectorstore(self, text_chunks: List[str]):
        """Create vector store from text chunks for RAG retrieval"""
        try:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            from langchain_community.vectorstores import FAISS

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            )

            # Prepare chunks with metadata
            documents = []
            metadatas = []

            for i, chunk in enumerate(text_chunks):
                if len(chunk.strip()) > 50:  # Only include substantial chunks
                    documents.append(chunk)
                    metadatas.append({
                        'chunk_id': i,
                        'length': len(chunk),
                        'type': 'text_chunk'
                    })

            if documents:
                self.vectorstore = FAISS.from_texts(documents, embeddings, metadatas=metadatas)
                self.document_chunks = documents
                logger.info(f"Created vector store with {len(documents)} chunks")
                return True
            else:
                logger.warning("No suitable chunks found for vector store")
                return False

        except Exception as e:
            logger.error(f"Error creating vector store: {e}")
            return False

    def rag_query(self, query: str, llm, k: int = 3) -> Dict[str, Any]:
        """Perform RAG query: retrieve relevant chunks + KG facts, then generate response"""
        self.stats['rag_queries'] += 1
        logger.info(f"Processing RAG query: '{query[:50]}...'")

        if not self.vectorstore:
            return {"error": "Vector store not initialized. Call prepare_vectorstore() first."}

        try:
            # 1. RETRIEVE: Get relevant text chunks
            relevant_docs = self.vectorstore.similarity_search(query, k=k)
            retrieved_chunks = [doc.page_content for doc in relevant_docs]

            # 2. EXTRACT: Get entities from query
            query_entities = self.extract_entities_advanced(query)

            # 3. RETRIEVE: Get KG facts for entities
            kg_facts = self.retrieve_kg_facts_enhanced(query_entities)
            kg_context = self.format_kg_context_enhanced(kg_facts)

            # 4. EXTRACT & ENRICH: Get locations from query
            query_locations = self.location_extractor.extract_locations_from_text(query)
            enriched_locations = {}
            for location_name in query_locations[:5]:
                location_info = self.location_extractor.enrich_location(location_name)
                if location_info:
                    enriched_locations[location_name] = location_info

            location_context = self.format_location_context(enriched_locations)

            # 5. AUGMENT: Create enhanced context for generation
            context_parts = [
                f"QUERY: {query}",
                f"\nRETRIEVED RELEVANT TEXT CHUNKS:",
                "\n" + "\n---\n".join(retrieved_chunks),
                f"\nKNOWLEDGE GRAPH CONTEXT:",
                kg_context,
                f"\nLOCATION CONTEXT:",
                location_context
            ]

            enhanced_context = "\n".join(context_parts)

            # 6. GENERATE: Create comprehensive response
            rag_prompt = f"""You are an expert historian with access to multiple knowledge sources. Answer the question using the provided context.

{enhanced_context}

INSTRUCTIONS:
1. Answer the question comprehensively using ALL available context
2. Cite specific information from the retrieved text chunks
3. Incorporate relevant knowledge graph facts to enhance your answer
4. Include precise location information and coordinates when relevant
5. If information conflicts between sources, mention this
6. Be specific about dates, places, people, and events
7. If you cannot answer completely, explain what information is missing

QUESTION: {query}

ANSWER:"""

            # 7. GENERATE: Get LLM response
            response = llm.invoke([HumanMessage(content=rag_prompt)])

            return {
                "query": query,
                "answer": response.content,
                "retrieved_chunks": retrieved_chunks,
                "entities_found": query_entities,
                "kg_facts_count": sum(len(facts) for facts in kg_facts.values()),
                "locations_found": list(enriched_locations.keys()),
                "sources": {
                    "text_chunks": len(retrieved_chunks),
                    "kg_sources": list(set(fact.source for facts in kg_facts.values() for fact in facts)),
                    "location_sources": list(set(loc.source for loc in enriched_locations.values()))
                }
            }

        except Exception as e:
            logger.error(f"RAG query failed: {e}")
            return {"error": str(e)}

    def interactive_rag_session(self, llm):
        """Start an interactive RAG session"""
        print("\n🤖 Starting Interactive RAG Session")
        print("Ask questions about your text. Type 'quit' to exit.")
        print("=" * 60)

        while True:
            try:
                query = input("\n❓ Your question: ").strip()

                if query.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye!")
                    break

                if not query:
                    continue

                print("\n🔍 Processing your question...")
                result = self.rag_query(query, llm)

                if "error" in result:
                    print(f"❌ Error: {result['error']}")
                    continue

                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")

            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue

# Utility functions
def load_api_key():
    """Llama via Ollama doesn't need an API key"""
    print("✅ Using local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Llama LLM"""
    try:
        llm = ChatOllama(model="llama3.2", temperature=0)
        print("✅ Llama LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        print("💡 Make sure Ollama is running: ollama serve")
        print("💡 And pull the model: ollama pull llama3.2")
        return None

def prepare_vectorstore_from_text(text: str, multi_kg_system):
    """Create vector store from text"""
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        from langchain_community.vectorstores import FAISS

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )

        # Split text into sentences for better retrieval
        sentences = re.split(r'[.!?]+', text)
        texts = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 20]

        if not texts:
            return None

        vectorstore = FAISS.from_texts(texts, embeddings)
        multi_kg_system.vectorstore = vectorstore
        multi_kg_system.document_chunks = texts
        print(f"📚 Vector store created with {len(texts)} text segments")
        return vectorstore
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def main():
    """Main function with chunking support and RAG capabilities - LLAMA VERSION"""
    print("🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)")

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    # Prepare vector store for RAG FIRST
    print("\n📚 Setting up RAG vector store...")
    vectorstore = prepare_vectorstore_from_text(domain_text, multi_kg_system)

    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    if token_count > 10000:  # Reduced for Llama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Extract events and create RDF
    all_turtle_outputs = []
    all_entities = set()

    print("\n🔄 Processing chunks for event extraction...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)

        if i < len(chunks):
            time.sleep(1)

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""

        final_output = prefixes + "# Historical Events with RAG-Enhanced Embedded Location Data (LLAMA)\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved enhanced RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        print(f"   - Location duplicates avoided: {multi_kg_system.stats['location_duplicates_avoided']}")
        print(f"   - Unique global locations: {len(multi_kg_system.global_locations)}")
        print(f"   - RAG queries for RDF generation: {multi_kg_system.stats['rag_queries']}")

        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")

        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")

        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    # START RAG SESSION HERE!
    if vectorstore:
        print(f"\n🤖 RAG System Ready!")

        # Show example queries
        print(f"\n💡 Try asking questions like:")
        print(f"   - 'What battles happened in Sicily?'")
        print(f"   - 'Who were the main leaders mentioned?'")
        print(f"   - 'What events occurred in 415 BC?'")
        print(f"   - 'Describe the naval engagements'")
        print(f"   - 'What was the outcome of the siege?'")

        # Ask if user wants to start interactive session
        response = input(f"\n❓ Start interactive RAG session? (y/n): ").strip()

        # Check if user typed a question instead of y/n
        if response.lower() not in ['y', 'yes', 'n', 'no', '']:
            # User typed a question directly!
            print(f"\n🔍 Processing your question: '{response}'")
            result = multi_kg_system.rag_query(response, llm)

            if "error" not in result:
                print(f"\n📝 **Answer:**")
                print(result['answer'])

                print(f"\n📊 **Sources Used:**")
                print(f"   - Text chunks: {result['sources']['text_chunks']}")
                print(f"   - KG sources: {result['sources']['kg_sources']}")
                print(f"   - Entities: {', '.join(result['entities_found'][:5])}")
                if result['locations_found']:
                    print(f"   - Locations: {', '.join(result['locations_found'])}")
            else:
                print(f"❌ Error: {result['error']}")

            # Ask if they want to continue with interactive session
            continue_response = input(f"\n❓ Continue with interactive RAG session? (y/n): ").strip().lower()
            if continue_response in ['y', 'yes', '']:
                multi_kg_system.interactive_rag_session(llm)

        elif response.lower() in ['y', 'yes', '']:
            multi_kg_system.interactive_rag_session(llm)
        else:
            print(f"\n💡 You can also query programmatically:")
            print(f"   result = multi_kg_system.rag_query('your question', llm)")

            # Offer a few sample queries
            sample_queries = [
                "What are the main events mentioned in the text?",
                "Which locations are mentioned?",
                "Who are the key people involved?"
            ]

            print(f"\n🔍 Running sample queries:")
            for query in sample_queries:
                print(f"\n❓ Sample query: '{query}'")
                result = multi_kg_system.rag_query(query, llm)
                if "error" not in result:
                    print(f"📝 Answer: {result['answer'][:200]}...")
                    print(f"📊 Sources: {len(result['retrieved_chunks'])} chunks, {result['kg_facts_count']} KG facts")
                else:
                    print(f"❌ Error: {result['error']}")
                print("-" * 40)

    else:
        print("⚠️  Could not create vector store for RAG functionality")

    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for RDF results.")

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Verifying Langchain Ollama import...
✅ ChatOllama successfully imported after reinstallation.
🚀 Starting Multi-Knowledge Graph RAG System with Chunking (LLAMA)
✅ Using local Ollama - no API key needed.
Loaded text from /content/drive/MyDrive/part_aa
📄 Using YOUR text from /content/drive/MyDrive/part_aa
📝 Text length: 398568 characters
✅ Llama LLM initialized successfully.

📚 Setting up RAG vector store...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📚 Vector store created with 1980 text segments
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 10 chunks

🔄 Processing chunks for event extraction...

🔄 Processing chunk 1/10...


This
This
Author
Author
Translator
Translator
Release Date
Release Date
Character
Character



🔄 Processing chunk 2/10...


...


🔄 Processing chunk 8/10...


Third Year
Third Year

The

The



🔄 Processing chunk 9/10...



🔄 Processing chunk 10/10...



✅ Saved enhanced RDF to /content/drive/MyDrive/extracted_events_rag_with_multi_kg_llama.ttl
📊 Processing Statistics:
   - Total chunks processed: 10
   - Successful chunks: 10
   - Unique entities found: 283
   - Total KG facts retrieved: 337
   - Cache hits: 618
   - Locations found: 97
   - Locations with coordinates: 97
   - Location duplicates avoided: 53
   - Unique global locations: 44
   - RAG queries for RDF generation: 10

🔗 Knowledge Graph Connector Statistics:
   - Wikidata: 59/72 requests (81.9% success)
   - DBpedia: 48/58 requests (82.8% success)
   - ConceptNet: 5/97 requests (5.2% success)
   - Location enrichment: 86/287 locations enriched (30.0% success)

📝 Sample of generated RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org

In [ ]:
"""
Enhanced Multi-Knowledge Graph System with Text Chunking - LLAMA VERSION
Handles large texts by processing them in chunks to avoid token limits
RAG FUNCTIONALITY DEACTIVATED (Knowledge Graph Enhancement Still Active)
"""

import os
import re
import time
import json
import hashlib
from typing import List, Dict, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

from rdflib import Graph, RDFS, RDF, OWL, URIRef, Namespace, Literal
from rdflib.namespace import XSD, SKOS

# Configuration - UPDATED FOR GOOGLE COLAB
INPUT_TEXT_FILE = "/content/drive/MyDrive/part_aa"  # Update this path as needed
ONTOLOGY_PATH = "/content/drive/MyDrive/wiki.owl"
LOCATION_ONTOLOGY_PATH = "/content/drive/MyDrive/locations.owl"
OUTPUT_RAG_TTL = '/content/drive/MyDrive/extracted_events_norag_with_multi_kg_llama.ttl'
OUTPUT_RAG_OWL = '/content/drive/MyDrive/extracted_events_norag_with_multi_kg_llama.owl'
KG_CACHE_FILE = '/content/drive/MyDrive/kg_cache.json'
LOCATION_CACHE_FILE = '/content/drive/MyDrive/location_cache.json'
KG_ANALYSIS_REPORT = '/content/drive/MyDrive/multi_kg_analysis_report.txt'

# Token limits - adjusted for Llama
MAX_TOKENS_PER_REQUEST = 50000  # Conservative limit for Llama
CHUNK_OVERLAP = 200  # Characters to overlap between chunks

# RAG DEACTIVATION FLAG
RAG_ENABLED = False  # Set to False to deactivate RAG

# Namespaces
EX = Namespace("http://example.org/")
STE = Namespace("http://www.example.org/ste#")
DBP = Namespace("http://dbpedia.org/ontology/")
LAC = Namespace("http://ontologia.fr/OTB/lac#")
WD = Namespace("http://www.wikidata.org/entity/")
YAGO = Namespace("http://yago-knowledge.org/resource/")
CN = Namespace("http://conceptnet.io/c/en/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
DBPR = Namespace("http://dbpedia.org/resource/")

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Imports
try:
    if RAG_ENABLED:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        from langchain_community.vectorstores import FAISS
    from langchain_ollama import ChatOllama  # Changed from OpenAI to Ollama
    from langchain.schema import HumanMessage
except ImportError as e:
    print(f"ImportError: {e}")
    print("Install dependencies: pip install rdflib langchain langchain-ollama langchain-community requests")
    exit(1)

@dataclass
class LocationInfo:
    """Location information with coordinates"""
    name: str
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    country: Optional[str] = None
    region: Optional[str] = None
    source: str = "extracted"
    confidence: float = 1.0
    uri: Optional[str] = None

@dataclass
class EnhancedKnowledgeFact:
    """Enhanced knowledge fact with metadata"""
    subject: str
    predicate: str
    object: str
    source: str
    confidence: float = 1.0
    context: Optional[str] = None
    temporal: Optional[str] = None
    spatial: Optional[str] = None
    evidence_score: float = 1.0
    source_uri: Optional[str] = None

class LocationExtractor:
    """Extracts and enriches location information"""

    def __init__(self, ontology_path: str = LOCATION_ONTOLOGY_PATH):
        self.ontology_path = ontology_path
        self.location_graph = None
        self.location_cache = self._load_location_cache()
        self.load_location_ontology()

    def _load_location_cache(self) -> Dict:
        """Load location cache"""
        if os.path.exists(LOCATION_CACHE_FILE):
            try:
                with open(LOCATION_CACHE_FILE, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load location cache: {e}")
        return {}

    def _save_location_cache(self):
        """Save location cache"""
        try:
            with open(LOCATION_CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.location_cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save location cache: {e}")

    def load_location_ontology(self):
        """Load locations.owl ontology"""
        try:
            if os.path.exists(self.ontology_path):
                self.location_graph = Graph()
                self.location_graph.parse(self.ontology_path, format="xml")
                logger.info(f"Loaded location ontology from {self.ontology_path}")
            else:
                logger.warning(f"Location ontology not found at {self.ontology_path}")
                self.location_graph = None
        except Exception as e:
            logger.error(f"Error loading location ontology: {e}")
            self.location_graph = None

    def extract_locations_from_text(self, text: str) -> List[str]:
        """Extract potential location names from text"""
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?=\s+(?:in|near|at|from|to))\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]

        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)

        location_stopwords = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If',
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Most',
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December', 'Monday', 'Tuesday',
            'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        }

        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and
                not loc.isdigit() and not re.match(r'^\d+', loc)):
                filtered_locations.append(loc)

        return list(set(filtered_locations))

    def get_location_from_ontology(self, location_name: str) -> Optional[LocationInfo]:
        """Get location info from local ontology"""
        if not self.location_graph:
            return None

        try:
            query = f"""
            SELECT DISTINCT ?location ?lat ?long ?country ?region WHERE {{
                ?location rdfs:label ?label .
                FILTER(regex(?label, "{location_name}", "i"))
                OPTIONAL {{ ?location geo:lat ?lat }}
                OPTIONAL {{ ?location geo:long ?long }}
                OPTIONAL {{ ?location dbp:country ?country }}
                OPTIONAL {{ ?location dbp:region ?region }}
            }}
            """

            results = self.location_graph.query(query)
            for row in results:
                return LocationInfo(
                    name=location_name,
                    latitude=float(row.lat) if row.lat else None,
                    longitude=float(row.long) if row.long else None,
                    country=str(row.country) if row.country else None,
                    region=str(row.region) if row.region else None,
                    source="local_ontology",
                    uri=str(row.location) if row.location else None
                )
        except Exception as e:
            logger.debug(f"Ontology query failed for {location_name}: {e}")

        return None

    def get_location_from_dbpedia(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from DBpedia"""
        try:
            time.sleep(0.5)
            entity_uri = f"http://dbpedia.org/resource/{location_name.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?lat ?long ?country ?region WHERE {{
                <{entity_uri}> geo:lat ?lat ;
                               geo:long ?long .
                OPTIONAL {{ <{entity_uri}> dbo:country ?country }}
                OPTIONAL {{ <{entity_uri}> dbo:region ?region }}
            }}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get("https://dbpedia.org/sparql", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                bindings = data.get('results', {}).get('bindings', [])

                if bindings:
                    binding = bindings[0]
                    return LocationInfo(
                        name=location_name,
                        latitude=float(binding.get('lat', {}).get('value', 0)),
                        longitude=float(binding.get('long', {}).get('value', 0)),
                        country=binding.get('country', {}).get('value', ''),
                        region=binding.get('region', {}).get('value', ''),
                        source="dbpedia",
                        uri=entity_uri
                    )

        except Exception as e:
            logger.debug(f"DBpedia location query failed for {location_name}: {e}")

        return None

    def get_location_from_wikidata(self, location_name: str) -> Optional[LocationInfo]:
        """Get location coordinates from Wikidata with disambiguation"""
        try:
            time.sleep(0.5)

            # Try multiple query strategies to get the right location
            queries = [
                # Try exact label match first
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """,
                # Try with additional filters for places/locations
                f"""
                SELECT DISTINCT ?item ?itemLabel ?coord ?country ?countryLabel WHERE {{
                  ?item rdfs:label "{location_name}"@en .
                  ?item wdt:P625 ?coord .
                  ?item wdt:P31/wdt:P279* wd:Q486972 .  # human settlement
                  OPTIONAL {{ ?item wdt:P17 ?country }}
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                LIMIT 5
                """
            ]

            for query in queries:
                params = {'query': query, 'format': 'json'}
                response = requests.get("https://query.wikidata.org/sparql", params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    bindings = data.get('results', {}).get('bindings', [])

                    if bindings:
                        # Prefer results with country information
                        best_binding = None
                        for binding in bindings:
                            if binding.get('country'):
                                best_binding = binding
                                break

                        if not best_binding:
                            best_binding = bindings[0]

                        coord_str = best_binding.get('coord', {}).get('value', '')

                        coord_match = re.search(r'Point\(([+-]?\d*\.?\d+)\s+([+-]?\d*\.?\d+)\)', coord_str)
                        if coord_match:
                            longitude = float(coord_match.group(1))
                            latitude = float(coord_match.group(2))

                            return LocationInfo(
                                name=location_name,
                                latitude=latitude,
                                longitude=longitude,
                                country=best_binding.get('countryLabel', {}).get('value', ''),
                                source="wikidata",
                                uri=best_binding.get('item', {}).get('value', '')
                            )

        except Exception as e:
            logger.debug(f"Wikidata location query failed for {location_name}: {e}")

        return None

    def validate_coordinates(self, location_info: LocationInfo) -> bool:
        """Validate that coordinates make sense for the location"""
        if not location_info.latitude or not location_info.longitude:
            return True

        lat, lon = location_info.latitude, location_info.longitude

        # Basic coordinate range validation
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            logger.warning(f"Invalid coordinates for {location_info.name}: {lat}, {lon}")
            return False

        return True

    def enrich_location(self, location_name: str) -> Optional[LocationInfo]:
        """Get enriched location information with coordinates"""
        if location_name in self.location_cache:
            cached = self.location_cache[location_name]
            return LocationInfo(**cached) if cached else None

        location_info = None

        location_info = self.get_location_from_ontology(location_name)

        if not location_info:
            location_info = self.get_location_from_wikidata(location_name)

        if not location_info:
            location_info = self.get_location_from_dbpedia(location_name)

        if location_info:
            self.location_cache[location_name] = {
                'name': location_info.name,
                'latitude': location_info.latitude,
                'longitude': location_info.longitude,
                'country': location_info.country,
                'region': location_info.region,
                'source': location_info.source,
                'confidence': location_info.confidence,
                'uri': location_info.uri
            }
        else:
            self.location_cache[location_name] = None

        self._save_location_cache()

        if location_info:
            self.validate_coordinates(location_info)

        return location_info

class TextChunker:
    """Handles text chunking to manage token limits for Llama"""

    def __init__(self, model_name: str = "llama3.2"):
        self.model_name = model_name

    def count_tokens(self, text: str) -> int:
        """Approximate token count for Llama (roughly 4 chars per token)"""
        return len(text) // 4

    def chunk_text_by_sentences(self, text: str, max_tokens: int = 10000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            test_chunk = current_chunk + " " + sentence if current_chunk else sentence

            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk

        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

class BaseKGConnector:
    """Base class for knowledge graph connectors"""

    def __init__(self, name: str, base_url: str, rate_limit: float = 1.0):
        self.name = name
        self.base_url = base_url
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.request_count = 0
        self.success_count = 0

    def _rate_limit_wait(self):
        """Enforce rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request_time = time.time()
        self.request_count += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get connector statistics"""
        return {
            'name': self.name,
            'requests': self.request_count,
            'successes': self.success_count,
            'success_rate': self.success_count / max(1, self.request_count)
        }

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Abstract method to retrieve facts"""
        raise NotImplementedError

class EnhancedWikidataConnector(BaseKGConnector):
    """Wikidata connector"""

    def __init__(self):
        super().__init__("Wikidata", "https://query.wikidata.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from Wikidata with timeout protection"""
        try:
            self._rate_limit_wait()

            sparql_query = f"""
            SELECT DISTINCT ?subject ?subjectLabel ?predicate ?predicateLabel ?object ?objectLabel WHERE {{
              {{
                ?subject ?label "{entity}"@en .
              }} UNION {{
                ?subject rdfs:label "{entity}"@en .
              }}

              ?subject ?predicate ?object .
              FILTER(?predicate != wdt:P31 && ?predicate != wdt:P279)

              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    fact = EnhancedKnowledgeFact(
                        subject=binding.get('subjectLabel', {}).get('value', entity),
                        predicate=binding.get('predicateLabel', {}).get('value', 'related_to'),
                        object=binding.get('objectLabel', {}).get('value', ''),
                        source=self.name,
                        confidence=0.9,
                        source_uri=binding.get('subject', {}).get('value')
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from Wikidata for '{entity}'")
                return facts
            else:
                logger.warning(f"Wikidata returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"Wikidata query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"Wikidata query failed for '{entity}': {e}")

        return []

class EnhancedDBpediaConnector(BaseKGConnector):
    """DBpedia connector"""

    def __init__(self):
        super().__init__("DBpedia", "https://dbpedia.org/sparql", 1.0)

    def retrieve_facts(self, entity: str, limit: int = 3) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from DBpedia with timeout protection"""
        try:
            self._rate_limit_wait()

            entity_uri = f"http://dbpedia.org/resource/{entity.replace(' ', '_')}"

            sparql_query = f"""
            SELECT DISTINCT ?predicate ?object WHERE {{
              <{entity_uri}> ?predicate ?object .
              FILTER(LANG(?object) = "en" || !isLiteral(?object))
              FILTER(!isBlank(?object))
            }}
            LIMIT {limit}
            """

            params = {'query': sparql_query, 'format': 'json'}
            response = requests.get(self.base_url, params=params, timeout=12)

            if response.status_code == 200:
                data = response.json()
                facts = []

                for binding in data.get('results', {}).get('bindings', []):
                    predicate = binding.get('predicate', {}).get('value', '')
                    obj = binding.get('object', {}).get('value', '')

                    predicate_name = predicate.split('/')[-1].replace('_', ' ')

                    fact = EnhancedKnowledgeFact(
                        subject=entity,
                        predicate=predicate_name,
                        object=obj,
                        source=self.name,
                        confidence=0.85,
                        source_uri=entity_uri
                    )
                    facts.append(fact)

                self.success_count += 1
                logger.info(f"Retrieved {len(facts)} facts from DBpedia for '{entity}'")
                return facts
            else:
                logger.warning(f"DBpedia returned status {response.status_code} for {entity}")

        except requests.Timeout:
            logger.warning(f"DBpedia query timeout for '{entity}'")
        except Exception as e:
            logger.warning(f"DBpedia query failed for '{entity}': {e}")

        return []

class EnhancedConceptNetConnector(BaseKGConnector):
    """ConceptNet connector with dynamic concept discovery"""

    def __init__(self):
        super().__init__("ConceptNet", "http://api.conceptnet.io", 0.5)

    def search_related_concepts(self, entity: str) -> List[str]:
        """Search for related concepts using ConceptNet's search API"""
        try:
            search_url = f"{self.base_url}/search?text={entity.replace(' ', '%20')}&limit=10"
            response = requests.get(search_url, timeout=10)

            related_concepts = []
            if response.status_code == 200:
                data = response.json()
                for edge in data.get('edges', []):
                    start = edge.get('start', {}).get('label', '')
                    end = edge.get('end', {}).get('label', '')

                    for concept_path in [start, end]:
                        if concept_path and '/c/en/' in concept_path:
                            concept = concept_path.replace('/c/en/', '').replace('_', ' ')
                            if concept.lower() != entity.lower() and len(concept) > 2:
                                related_concepts.append(concept)

            return list(set(related_concepts))[:5]

        except Exception as e:
            logger.debug(f"ConceptNet search failed for {entity}: {e}")
            return []

    def query_concept_directly(self, concept: str, limit: int = 20) -> List[dict]:
        """Query a specific concept and return raw edges"""
        try:
            concept_path = f"/c/en/{concept.lower().replace(' ', '_')}"
            url = f"{self.base_url}{concept_path}?limit={limit}"

            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data.get('edges', [])

        except Exception as e:
            logger.debug(f"ConceptNet direct query failed for {concept}: {e}")

        return []

    def retrieve_facts(self, entity: str, limit: int = 100) -> List[EnhancedKnowledgeFact]:
        """Retrieve facts from ConceptNet through dynamic discovery"""
        try:
            self._rate_limit_wait()
            all_facts = []

            direct_edges = self.query_concept_directly(entity, limit//2)
            related_concepts = self.search_related_concepts(entity)

            for edge in direct_edges:
                fact = self._edge_to_fact(edge, entity, "direct")
                if fact:
                    all_facts.append(fact)

            for concept in related_concepts:
                concept_edges = self.query_concept_directly(concept, 5)
                for edge in concept_edges:
                    fact = self._edge_to_fact(edge, entity, f"via_{concept}")
                    if fact:
                        all_facts.append(fact)

            if all_facts:
                self.success_count += 1
                logger.info(f"Retrieved {len(all_facts)} facts from ConceptNet for '{entity}'")
                if related_concepts:
                    logger.info(f"  - Found related concepts: {related_concepts}")

            return all_facts[:limit]

        except Exception as e:
            logger.error(f"ConceptNet query failed for '{entity}': {e}")

        return []

    def _edge_to_fact(self, edge: dict, original_entity: str, discovery_method: str) -> Optional[EnhancedKnowledgeFact]:
        """Convert ConceptNet edge to EnhancedKnowledgeFact"""
        try:
            start = edge.get('start', {})
            end = edge.get('end', {})
            relation = edge.get('rel', {})
            weight = edge.get('weight', 1.0)

            start_label = start.get('label', '').replace('/c/en/', '').replace('_', ' ')
            end_label = end.get('label', '').replace('/c/en/', '').replace('_', ' ')
            rel_label = relation.get('label', 'related_to')

            if not start_label or not end_label or len(start_label) < 2 or len(end_label) < 2:
                return None

            confidence_multiplier = 1.0 if discovery_method == "direct" else 0.6

            return EnhancedKnowledgeFact(
                subject=original_entity,
                predicate=rel_label,
                object=end_label if start_label.lower() in original_entity.lower() else start_label,
                source=self.name,
                confidence=min(weight * confidence_multiplier, 1.0),
                context=f"Discovered {discovery_method}"
            )

        except Exception as e:
            logger.debug(f"Error converting edge to fact: {e}")
            return None

class MultiKGCache:
    """Caching system for knowledge graph facts"""

    def __init__(self, cache_file: str = KG_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self) -> Dict:
        """Load cache from file"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                logger.warning(f"Could not load cache: {e}")
        return {}

    def _save_cache(self):
        """Save cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logger.warning(f"Could not save cache: {e}")

    def get_cache_key(self, source: str, entity: str) -> str:
        """Generate cache key"""
        return f"{source}:{hashlib.md5(entity.encode()).hexdigest()}"

    def get(self, source: str, entity: str) -> Optional[List[Dict]]:
        """Get cached facts"""
        key = self.get_cache_key(source, entity)
        return self.cache.get(key)

    def set(self, source: str, entity: str, facts: List[EnhancedKnowledgeFact]):
        """Cache facts"""
        key = self.get_cache_key(source, entity)
        serializable_facts = []
        for fact in facts:
            serializable_facts.append({
                'subject': fact.subject,
                'predicate': fact.predicate,
                'object': fact.object,
                'source': fact.source,
                'confidence': fact.confidence,
                'context': fact.context,
                'temporal': fact.temporal,
                'spatial': fact.spatial,
                'evidence_score': fact.evidence_score,
                'source_uri': fact.source_uri
            })
        self.cache[key] = serializable_facts
        self._save_cache()

class EnhancedMultiKGRAGSystem:
    """Multi-Knowledge Graph system with RAG functionality DEACTIVATED - LLAMA VERSION"""

    def __init__(self):
        self.connectors = {
            'wikidata': EnhancedWikidataConnector(),
            'dbpedia': EnhancedDBpediaConnector(),
            'conceptnet': EnhancedConceptNetConnector()
        }
        self.cache = MultiKGCache()
        self.chunker = TextChunker()
        self.location_extractor = LocationExtractor()
        self.global_locations = {}
        # RAG components deactivated
        self.vectorstore = None
        self.document_chunks = []
        self.stats = {
            'queries_processed': 0,
            'entities_extracted': 0,
            'facts_retrieved': 0,
            'cache_hits': 0,
            'chunks_processed': 0,
            'locations_found': 0,
            'locations_with_coordinates': 0,
            'location_duplicates_avoided': 0,
            'rag_queries': 0
        }

    def extract_entities_advanced(self, text: str) -> List[str]:
        """Extract entities from text"""
        entities = []

        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        entities.extend(matches)

        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 'Few', 'Most', 'Each', 'Every',
            'First', 'Second', 'Third', 'Last', 'Next', 'Previous', 'Before', 'After', 'During'
        }

        filtered_entities = []
        for entity in entities:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)

        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)

        return unique_entities[:15]

    def retrieve_kg_facts_enhanced(self, entities: List[str]) -> Dict[str, List[EnhancedKnowledgeFact]]:
        """Retrieve facts from knowledge graphs with improved timeout handling"""
        all_facts = {}
        cache_hits = 0

        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = {}

            for entity in entities:
                for source_name, connector in self.connectors.items():
                    # Check cache first
                    cached_facts = self.cache.get(source_name, entity)
                    if cached_facts:
                        cache_hits += 1
                        if entity not in all_facts:
                            all_facts[entity] = []
                        for fact_data in cached_facts:
                            fact = EnhancedKnowledgeFact(**fact_data)
                            all_facts[entity].append(fact)
                    else:
                        future = executor.submit(connector.retrieve_facts, entity, 3)
                        futures[future] = (entity, source_name)

            # Collect results with better timeout handling
            completed = 0
            total_futures = len(futures)

            try:
                for future in as_completed(futures, timeout=45):
                    entity, source_name = futures[future]
                    completed += 1

                    try:
                        facts = future.result(timeout=5)
                        if facts:
                            self.cache.set(source_name, entity, facts)

                            if entity not in all_facts:
                                all_facts[entity] = []
                            all_facts[entity].extend(facts)

                            self.stats['facts_retrieved'] += len(facts)

                        logger.debug(f"✅ {source_name} completed for {entity} ({completed}/{total_futures})")

                    except Exception as e:
                        logger.warning(f"❌ {source_name} failed for {entity}: {e}")
                        continue

            except TimeoutError:
                pending_count = total_futures - completed
                logger.warning(f"⏰ Timeout: {pending_count}/{total_futures} KG queries still pending, continuing with available results")

                # Cancel remaining futures
                for future in futures:
                    if not future.done():
                        future.cancel()

        self.stats['cache_hits'] += cache_hits
        logger.info(f"KG retrieval completed: {completed}/{total_futures} successful, {cache_hits} cache hits")
        return all_facts

    def format_kg_context_enhanced(self, kg_facts: Dict[str, List[EnhancedKnowledgeFact]]) -> str:
        """Format KG facts into context string"""
        context_parts = []

        for entity, facts in kg_facts.items():
            if facts:
                sorted_facts = sorted(facts, key=lambda f: f.confidence, reverse=True)

                context_parts.append(f"\n=== Knowledge about {entity} ===")

                by_source = {}
                for fact in sorted_facts[:3]:
                    if fact.source not in by_source:
                        by_source[fact.source] = []
                    by_source[fact.source].append(fact)

                for source, source_facts in by_source.items():
                    context_parts.append(f"\nFrom {source}:")
                    for fact in source_facts[:2]:
                        fact_str = f"- {fact.subject} {fact.predicate} {fact.object}"
                        if fact.confidence < 0.8:
                            fact_str += f" (confidence: {fact.confidence:.2f})"
                        context_parts.append(fact_str)

        return "\n".join(context_parts)

    def register_global_location(self, location_info: LocationInfo) -> str:
        """Register location globally and return unique identifier"""
        location_key = location_info.name.lower().strip()

        if location_key in self.global_locations:
            existing = self.global_locations[location_key]
            if (location_info.latitude and location_info.longitude and
                (not existing.latitude or not existing.longitude)):
                self.global_locations[location_key] = location_info
                logger.info(f"Updated coordinates for {location_info.name}")
            else:
                self.stats['location_duplicates_avoided'] += 1
                logger.debug(f"Location {location_info.name} already registered")
        else:
            self.global_locations[location_key] = location_info
            logger.info(f"Registered new location: {location_info.name}")

        clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
        return f"ste:Location_{clean_name}"

    def process_chunk(self, chunk: str, chunk_num: int, llm) -> str:
        """Process a single chunk of text WITHOUT RAG (RAG DEACTIVATED) - LLAMA VERSION"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars) - RAG DISABLED")

        # RAG retrieval DEACTIVATED - skip this step
        # relevant_context = ""

        # Extract entities and locations (this remains the same)
        entities = self.extract_entities_advanced(chunk)
        locations = self.location_extractor.extract_locations_from_text(chunk)
        logger.info(f"Found potential locations in chunk {chunk_num}: {locations}")

        # Enrich locations with coordinates
        enriched_locations = {}
        for location_name in locations[:10]:
            location_info = self.location_extractor.enrich_location(location_name)
            if location_info:
                self.register_global_location(location_info)
                enriched_locations[location_name] = location_info
                self.stats['locations_found'] += 1
                if location_info.latitude and location_info.longitude:
                    self.stats['locations_with_coordinates'] += 1

        if not entities and not enriched_locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""

        logger.info(f"Found entities in chunk {chunk_num}: {entities[:5]}...")
        logger.info(f"Enriched {len(enriched_locations)} locations with coordinates")

        # Get KG facts for entities (this remains the same)
        kg_facts = self.retrieve_kg_facts_enhanced(entities)
        kg_context = self.format_kg_context_enhanced(kg_facts)
        location_context = self.format_location_context(enriched_locations)

        # SIMPLIFIED PROMPT WITHOUT RAG - OPTIMIZED FOR LLAMA
        simplified_prompt = f"""You are extracting historical events from text chunks. Use knowledge graph facts and location coordinates to enhance your extraction.

CURRENT TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

KNOWLEDGE GRAPH FACTS FOR ENTITIES IN THIS CHUNK:
{kg_context}

LOCATION INFORMATION WITH COORDINATES:
{location_context}

TASK: Extract ONLY the events that are actually mentioned in the current text chunk.

Requirements:
1. Extract ONLY events mentioned in the CURRENT text chunk
2. Use KG facts to enhance entity information
3. Use location coordinates to provide precise geographical data
4. Include ALL these properties for each event:
   - ste:hasType (description of event)
   - ste:hasAgent (who caused/led the event)
   - ste:hasTime (when it happened)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (latitude coordinate if available)
   - ste:hasLongitude (longitude coordinate if available)
   - ste:hasCountry (country if available)
   - ste:hasRegion (region if available)
   - ste:hasLocationSource (source of coordinates: wikidata/dbpedia/local_ontology)
   - ste:hasResult (outcome/consequence)

Output format (do not include prefixes, they will be added later):
```turtle
ste:Event{chunk_num}_1 a ste:Event, dbp:SpecificEventType ;
    ste:hasType "specific description from current chunk" ;
    ste:hasAgent "specific person from current chunk" ;
    ste:hasTime "specific date from current chunk" ;
    ste:hasLocation "specific location from current chunk" ;
    ste:hasLatitude "37.1234"^^xsd:double ;
    ste:hasLongitude "15.5678"^^xsd:double ;
    ste:hasCountry "Italy" ;
    ste:hasRegion "Sicily" ;
    ste:hasLocationSource "wikidata" ;
    ste:hasResult "specific outcome from current chunk" .
```

IMPORTANT:
- Extract events ONLY from the CURRENT text chunk
- Use KG facts to enrich entity details
- Include precise coordinates from location sources
- Only extract events explicitly mentioned in the current chunk
- If no clear events are found in current chunk, return empty
"""

        try:
            response = llm.invoke([HumanMessage(content=simplified_prompt)])
            turtle_output = self.clean_turtle(response.content)
            self.stats['chunks_processed'] += 1
            logger.info(f"Generated RDF for chunk {chunk_num} (without RAG)")
            return turtle_output
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num}: {e}")
            return ""

    def format_location_context(self, enriched_locations: Dict[str, LocationInfo]) -> str:
        """Format location information into context string"""
        if not enriched_locations:
            return "No location coordinates available."

        context_parts = ["\n=== Location Information ==="]

        for location_name, location_info in enriched_locations.items():
            context_parts.append(f"\n{location_name}:")
            context_parts.append(f"  - Source: {location_info.source}")

            if location_info.latitude and location_info.longitude:
                context_parts.append(f"  - Coordinates: {location_info.latitude}, {location_info.longitude}")
                if location_info.source == "corrected":
                    context_parts.append(f"  - NOTE: Coordinates were corrected for historical accuracy")
            else:
                context_parts.append("  - Coordinates: Not available")

            if location_info.country:
                context_parts.append(f"  - Country: {location_info.country}")

            if location_info.region:
                context_parts.append(f"  - Region: {location_info.region}")

            if location_info.uri:
                context_parts.append(f"  - URI: {location_info.uri}")

        return "\n".join(context_parts)

    def generate_global_location_rdf(self) -> str:
        """Generate RDF for all unique locations found across all chunks"""
        if not self.global_locations:
            return ""

        location_rdf_parts = []

        for location_key, location_info in self.global_locations.items():
            clean_name = re.sub(r'[^a-zA-Z0-9]', '', location_info.name)
            location_id = f"ste:Location_{clean_name}"

            rdf_lines = [f'{location_id} a ste:Location ;']
            rdf_lines.append(f'    rdfs:label "{location_info.name}" ;')

            if location_info.latitude and location_info.longitude:
                rdf_lines.append(f'    geo:lat "{location_info.latitude}"^^xsd:double ;')
                rdf_lines.append(f'    geo:long "{location_info.longitude}"^^xsd:double ;')

            if location_info.country:
                rdf_lines.append(f'    ste:hasCountry "{location_info.country}" ;')

            if location_info.region:
                rdf_lines.append(f'    ste:hasRegion "{location_info.region}" ;')

            if location_info.source:
                rdf_lines.append(f'    ste:hasSource "{location_info.source}" ;')

            if location_info.uri:
                rdf_lines.append(f'    ste:hasURI <{location_info.uri}> ;')

            if rdf_lines[-1].endswith(' ;'):
                rdf_lines[-1] = rdf_lines[-1][:-2] + ' .'

            location_rdf_parts.append('\n'.join(rdf_lines))

        return '\n\n'.join(location_rdf_parts)

    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()

        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or
                stripped.startswith(':') or stripped.startswith('_') or
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)

        return '\n'.join(turtle_lines)

    # RAG METHODS DEACTIVATED
    def prepare_vectorstore(self, text_chunks: List[str]):
        """RAG DEACTIVATED: Vector store preparation disabled"""
        logger.info("RAG functionality is DEACTIVATED - vectorstore not created")
        return False

    def rag_query(self, query: str, llm, k: int = 20) -> Dict[str, Any]:
        """RAG DEACTIVATED: RAG queries disabled"""
        return {"error": "RAG functionality is DEACTIVATED. Set RAG_ENABLED=True to enable RAG features."}

    def interactive_rag_session(self, llm):
        """RAG DEACTIVATED: Interactive RAG session disabled"""
        print("\n❌ RAG functionality is DEACTIVATED")
        print("To enable RAG, set RAG_ENABLED=True at the top of the script")

# Utility functions
def load_api_key():
    """Llama via Ollama doesn't need an API key"""
    print("✅ Using local Ollama - no API key needed.")
    return "local"

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize Llama LLM"""
    try:
        llm = ChatOllama(model="llama3.2", temperature=0)
        print("✅ Llama LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        print("💡 Make sure Ollama is running: ollama serve")
        print("💡 And pull the model: ollama pull llama3.2")
        return None

def prepare_vectorstore_from_text(text: str, multi_kg_system):
    """RAG DEACTIVATED: Vector store creation disabled"""
    if not RAG_ENABLED:
        logger.info("RAG functionality is DEACTIVATED - vectorstore not created")
        return None

    # Original code would go here if RAG_ENABLED was True
    return None

def main():
    """Main function with chunking support (RAG DEACTIVATED) - LLAMA VERSION"""
    print("🚀 Starting Multi-Knowledge Graph System with Chunking (RAG DEACTIVATED) - LLAMA")

    api_key = load_api_key()
    if not api_key:
        return

    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC.
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes.
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using YOUR text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")

    multi_kg_system = EnhancedMultiKGRAGSystem()
    llm = initialize_llm(api_key)

    if not llm:
        return

    # Vector store preparation SKIPPED (RAG deactivated)
    print("\n❌ RAG vector store setup SKIPPED (RAG is DEACTIVATED)")

    token_count = multi_kg_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")

    if token_count > 10000:  # Reduced for Llama
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = multi_kg_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=10000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]

    # Extract events and create RDF (without RAG)
    all_turtle_outputs = []
    all_entities = set()

    print("\n🔄 Processing chunks for event extraction (without RAG)...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")

        turtle_output = multi_kg_system.process_chunk(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)

        chunk_entities = multi_kg_system.extract_entities_advanced(chunk)
        all_entities.update(chunk_entities)

        if i < len(chunks):
            time.sleep(1)

    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dbp: <http://dbpedia.org/ontology/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix dbpr: <http://dbpedia.org/resource/> .

"""

        final_output = prefixes + "# Historical Events with Knowledge Graph Enhanced Location Data (RAG DEACTIVATED) - LLAMA\n" + "\n\n".join(all_turtle_outputs)

        with open(OUTPUT_RAG_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)

        print(f"\n✅ Saved RDF to {OUTPUT_RAG_TTL}")
        print(f"📊 Processing Statistics:")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Unique entities found: {len(all_entities)}")
        print(f"   - Total KG facts retrieved: {multi_kg_system.stats['facts_retrieved']}")
        print(f"   - Cache hits: {multi_kg_system.stats['cache_hits']}")
        print(f"   - Locations found: {multi_kg_system.stats['locations_found']}")
        print(f"   - Locations with coordinates: {multi_kg_system.stats['locations_with_coordinates']}")
        print(f"   - Location duplicates avoided: {multi_kg_system.stats['location_duplicates_avoided']}")
        print(f"   - Unique global locations: {len(multi_kg_system.global_locations)}")
        print(f"   - RAG status: DEACTIVATED")
        print(f"   - LLM: Llama 3.2 via Ollama")

        print(f"\n🔗 Knowledge Graph Connector Statistics:")
        for name, connector in multi_kg_system.connectors.items():
            stats = connector.get_stats()
            print(f"   - {stats['name']}: {stats['successes']}/{stats['requests']} requests ({stats['success_rate']:.1%} success)")

        if multi_kg_system.location_extractor.location_cache:
            successful_locations = sum(1 for v in multi_kg_system.location_extractor.location_cache.values() if v is not None)
            total_locations = len(multi_kg_system.location_extractor.location_cache)
            print(f"   - Location enrichment: {successful_locations}/{total_locations} locations enriched ({successful_locations/total_locations:.1%} success)")

        print(f"\n📝 Sample of generated RDF:")
        print("="*60)
        print(final_output[:1000] + "..." if len(final_output) > 1000 else final_output)
        print("="*60)

    else:
        print("❌ No events were extracted from any chunks")

    # RAG SESSION DEACTIVATED
    print(f"\n❌ RAG System is DEACTIVATED")
    print(f"💡 To enable RAG functionality:")
    print(f"   1. Set RAG_ENABLED = True at the top of the script")
    print(f"   2. Ensure langchain dependencies are installed")
    print(f"   3. Re-run the script")

    print(f"\n🎉 Process complete! Check {OUTPUT_RAG_TTL} for RDF results.")
    print(f"📊 System ran in NON-RAG mode - only Knowledge Graph and Location enrichment was used.")
    print(f"🦙 Using Llama 3.2 via Ollama for processing.")

if __name__ == '__main__':
    main()

🚀 Starting Multi-Knowledge Graph System with Chunking (RAG DEACTIVATED) - LLAMA
✅ Using local Ollama - no API key needed.
Loaded text from /content/drive/MyDrive/part_aa
📄 Using YOUR text from /content/drive/MyDrive/part_aa
📝 Text length: 398568 characters
✅ Llama LLM initialized successfully.

❌ RAG vector store setup SKIPPED (RAG is DEACTIVATED)
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 10 chunks

🔄 Processing chunks for event extraction (without RAG)...

🔄 Processing chunk 1/10...


This
This
Author
Author
Translator
Translator
Release Date
Release Date
Character
Character



🔄 Processing chunk 2/10...

🔄 Processing chunk 3/10...


Congress
Congress

The Athenians

The Athenians



🔄 Processing chunk 4/10...

🔄 Processing chunk 5/10...

🔄 Processing chunk 6/10...

🔄 Processing chunk 7/10...


Second Year
Second Year



🔄 Processing chunk 8/10...


Third Year
Third Year

The

The



🔄 Processing chunk 9/10...

🔄 Processing chunk 10/10...

✅ Saved RDF to /content/drive/MyDrive/extracted_events_norag_with_multi_kg_llama.ttl
📊 Processing Statistics:
   - Total chunks processed: 10
   - Successful chunks: 10
   - Unique entities found: 123
   - Total KG facts retrieved: 1
   - Cache hits: 314
   - Locations found: 37
   - Locations with coordinates: 37
   - Location duplicates avoided: 21
   - Unique global locations: 16
   - RAG status: DEACTIVATED
   - LLM: Llama 3.2 via Ollama

🔗 Knowledge Graph Connector Statistics:
   - Wikidata: 36/46 requests (78.3% success)
   - DBpedia: 26/36 requests (72.2% success)
   - ConceptNet: 1/54 requests (1.9% success)
   - Location enrichment: 86/287 locations enriched (30.0% success)

📝 Sample of generated RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix d